In [1]:
import pandas as pd
import os
import time
import numpy as np
import tensorflow as tf
import numpy as np
import os
import random
import copy
import re
import csv
from collections import deque
from glob import glob
import bcolz
from sklearn.metrics import log_loss
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True,gpu_options=gpu_options))
%pwd

'C:\\Users\\chapanda\\OneDrive - Epsilon\\cp\\ACG\\03_Practise\\RnD\\crosssell'

In [2]:
path_out = 'C:\\Users\\chapanda\\data\\np\\'
#path_out = u'/cat/home/ubuntu/ikrt/mycode/np/'
N = 100
debug = False
batch_size = 128
num_training_steps = 1000000
learning_rate = 0.0003
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 500000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 50000
log_interval = 1000
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = False
dropout = 0

dilations=[2**i for i in range(6)]
dilations=dilations+dilations
filter_widths=[2]*6
filter_widths=filter_widths+filter_widths
skip_channels=124
residual_channels=256

prediction_dir = path_out + "predictions"
checkpoint_dir = path_out + "checkpoints"

if not os.path.isdir(prediction_dir):
    os.makedirs(prediction_dir)

if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)


In [3]:
start_time = 0
def tic():
    global start_time
    start_time = time.time()
def toc():
    global start_time
    elapsed_time = (time.time() - start_time)
    print("took me " + str(round(elapsed_time, 3))+" seconds to do this..")

In [4]:
def shape(tensor, dim=None):
    """Get tensor shape/dimension as list/int"""
    if dim is None:
        return tensor.shape.as_list()
    else:
        return tensor.shape.as_list()[dim]

In [5]:
class dataset(object):

    def __init__(self, path_out, suffix):
        
        self.filenames = [
            'user_id',
            'master_aisle_id',
            'master_department_id',
            'IsInOrder_history',
            'NextInOrder_history',
            'NumProductsFromDep_history',
            'OrderSize_history',
            'IndexInOrder_history',
            'order_number_history',
            'order_hour_of_day_history',
            'order_dow_history',
            'days_since_prior_order_history',
            'history_length',
            #'ProductID1_history_a',
            #'ProductID2_history_a',
            #'ProductID3_history_a',
            #'ProductID4_history_a',
            #'ProductID5_history_a',
            #'ProductID6_history_a',
            #'ProductID7_history_a',
            #'ProductID8_history_a',
            #'ProductID9_history_a',
            #'ProductID10_history_a',
            #'ProductID11_history_a',
            #'ProductID12_history_a',
            #'ProductID13_history_a',
            #'ProductID14_history_a',
            #'ProductID15_history_a',
            #'ProductID16_history_a',
            #'ProductID17_history_a',
            #'ProductID18_history_a',   
            #'ProductID19_history_a',
            #'ProductID20_history_a',
            'ProductID1_history',
            'ProductID2_history',
            'ProductID3_history',
            'ProductID4_history',
            'ProductID5_history',
            'ProductID6_history',
            'ProductID7_history',
            'ProductID8_history',
            'ProductID9_history',
            'ProductID10_history',
            'ProductID11_history',
            'ProductID12_history',
            'ProductID13_history',
            'ProductID14_history',
            'ProductID15_history',
            'ProductID16_history',
            'ProductID17_history',
            'ProductID18_history',   
            'ProductID19_history',
            'ProductID20_history',
            #'ProductNameEmbedding_history',
            'label'
        ]
        
        self.ids = 0
        self.start = 0
        self.dataFiles = glob(path_out+'user_id*.dat') 
        self.num_chunks = len(self.dataFiles)
        self.chunks_index = 0
        
        self.start = 0 
        self.arrays = [(bcolz.open(self.dataFiles[0].replace('user_id',i))[:]) for i in self.filenames]
        self.ids = np.arange(0,len(self.arrays[0]))

    def next_batch(self, batch_size, shuffle=True, is_test=False):
        while(1):
            if (self.arrays[0].shape[0] - self.start) >= batch_size:
                batch_ids = self.ids[self.start: (self.start + batch_size)]
                self.start += batch_size
                yield {globals()[placeholder] : copy.copy(array[batch_ids]) for placeholder, array in zip(self.filenames,self.arrays)} 
            else:
                self.start = 0 
                self.chunks_index += 1
                if self.chunks_index >= self.num_chunks:
                    self.chunks_index = 0
                self.arrays = [(bcolz.open(self.dataFiles[self.chunks_index].replace('user_id',i))[:]) for i in self.filenames]
                self.ids = np.arange(0,len(self.arrays[0]))
                if shuffle:
                    random.shuffle(self.ids)

In [6]:
train = dataset(path_out,"")

In [7]:
def sequence_log_loss(y, y_hat, sequence_lengths, max_sequence_length, eps=1e-15):
    """
    Calculates average log loss on variable length sequences.

    Args:
        y: Label tensor of shape [batch size, max_sequence_length, input units].
        y_hat: Prediction tensor, same shape as y.
        sequence_lengths: Sequence lengths.  Tensor of shape [batch_size].
        max_sequence_length: maximum length of padded sequence tensor.

    Returns:
        Log loss. 0-dimensional tensor.
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.minimum(tf.maximum(y_hat, eps), 1.0 - eps)
    log_losses = y*tf.log(y_hat) + (1.0 - y)*tf.log(1.0 - y_hat)
    sequence_mask = tf.cast(tf.sequence_mask(sequence_lengths, maxlen=max_sequence_length), tf.float32)
    avg_log_loss = -tf.reduce_sum(log_losses*sequence_mask) / tf.cast(tf.reduce_sum(sequence_lengths), tf.float32)
    return avg_log_loss

In [8]:
def temporal_convolution_layer(inputs, output_units, convolution_width, causal=False, dilation_rate=[1], bias=True,
                               activation=None, dropout=None, scope='temporal-convolution-layer', reuse=False):
    """
    Convolution over the temporal axis of sequence data.

    Args:
        inputs: Tensor of shape [batch size, max sequence length, input_units].
        output_units: Output channels for convolution.
        convolution_width: Number of timesteps to use in convolution.
        causal: Output at timestep t is a function of inputs at or before timestep t.
        dilation_rate:  Dilation rate along temporal axis.

    Returns:
        Tensor of shape [batch size, max sequence length, output_units].

    """
    with tf.variable_scope(scope, reuse=reuse):
        if causal:
            shift = int((convolution_width / 2) + (int(dilation_rate[0] - 1) / 2)) #FIXED THIS
            pad = tf.zeros([tf.shape(inputs)[0], shift, inputs.shape.as_list()[2]])
            inputs = tf.concat([pad, inputs], axis=1)

        W = tf.get_variable(
            name='weights',
            initializer=tf.contrib.layers.variance_scaling_initializer(),
            shape=[convolution_width, shape(inputs, 2), output_units]
        )

        z = tf.nn.convolution(inputs, W, padding='SAME', dilation_rate=dilation_rate)
        if bias:
            b = tf.get_variable(
                name='biases',
                initializer=tf.constant_initializer(),
                shape=[output_units]
            )
            z = z + b
        z = activation(z) if activation else z
        z = tf.nn.dropout(z, dropout) if dropout is not None else z
        z = z[:, :-shift, :] if causal else z
        return z


In [9]:
def wavenet(x, dilations, filter_widths, skip_channels, residual_channels, scope='wavenet', reuse=False):
    """
    A stack of causal dilated convolutions with paramaterized residual and skip connections as described
    in the WaveNet paper (with some minor differences).

    Args:
        x: Input tensor of shape [batch size, max sequence length, input units].
        dilations: List of dilations for each layer.  len(dilations) is the number of layers
        filter_widths: List of filter widths.  Same length as dilations.
        skip_channels: Number of channels to use for skip connections.
        residual_channels: Number of channels to use for residual connections.

    Returns:
        Tensor of shape [batch size, max sequence length, len(dilations)*skip_channels].
    """
    with tf.variable_scope(scope, reuse=reuse):

        # wavenet uses 2x1 conv here
        inputs = time_distributed_dense_layer(x, residual_channels, activation=tf.nn.tanh, scope='x-proj')

        skip_outputs = []
        for i, (dilation, filter_width) in enumerate(zip(dilations, filter_widths)):
            dilated_conv = temporal_convolution_layer(
                inputs=inputs,
                output_units=2*residual_channels,
                convolution_width=filter_width,
                causal=True,# CHECK THIS OUT
                dilation_rate=[dilation],
                scope='cnn-{}'.format(i) 
            )
            conv_filter, conv_gate = tf.split(dilated_conv, 2, axis=2)
            dilated_conv = tf.nn.tanh(conv_filter)*tf.nn.sigmoid(conv_gate)

            output_units = skip_channels + residual_channels
            outputs = time_distributed_dense_layer(dilated_conv, output_units, scope='cnn-{}-proj'.format(i))
            skips, residuals = tf.split(outputs, [skip_channels, residual_channels], axis=2)

            inputs += residuals
            skip_outputs.append(skips)

        skip_outputs = tf.nn.relu(tf.concat(skip_outputs, axis=2))
        return skip_outputs


In [10]:
def lstm_layer(inputs, lengths, state_size, keep_prob=1.0, scope='lstm-layer', reuse=False, return_final_state=False):
    """
    LSTM layer.

    Args:
        inputs: Tensor of shape [batch size, max sequence length, ...].
        lengths: Tensor of shape [batch size].
        state_size: LSTM state size.
        keep_prob: 1 - p, where p is the dropout probability.

    Returns:
        Tensor of shape [batch size, max sequence length, state_size] containing the lstm
        outputs at each timestep.

    """
    with tf.variable_scope(scope, reuse=reuse):
        cell_fw = tf.contrib.rnn.DropoutWrapper(
            tf.contrib.rnn.LSTMCell(
                state_size,
                reuse=reuse
            ),
            output_keep_prob=keep_prob
        )
        outputs, output_state = tf.nn.dynamic_rnn(
            inputs=inputs,
            cell=cell_fw,
            sequence_length=lengths,
            dtype=tf.float32
        )
        if return_final_state:
            return outputs, output_state
        else:
            return outputs

In [11]:
def time_distributed_dense_layer(inputs, output_units, bias=True, activation=None, batch_norm=None,
                                 dropout=None, scope='time-distributed-dense-layer', reuse=False):
    """
    Applies a shared dense layer to each timestep of a tensor of shape [batch_size, max_seq_len, input_units]
    to produce a tensor of shape [batch_size, max_seq_len, output_units].

    Args:
        inputs: Tensor of shape [batch size, max sequence length, ...].
        output_units: Number of output units.
        activation: activation function.
        dropout: dropout keep prob.

    Returns:
        Tensor of shape [batch size, max sequence length, output_units].

    """
    with tf.variable_scope(scope, reuse=reuse):
        W = tf.get_variable(
            name='weights',
            initializer=tf.contrib.layers.variance_scaling_initializer(),
            shape=[shape(inputs, -1), output_units]
        )
        z = tf.einsum('ijk,kl->ijl', inputs, W)
        if bias:
            b = tf.get_variable(
                name='biases',
                initializer=tf.constant_initializer(),
                shape=[output_units]
            )
            z = z + b

        if batch_norm is not None:
            z = tf.layers.batch_normalization(z, training=batch_norm, reuse=reuse)

        z = activation(z) if activation else z
        z = tf.nn.dropout(z, dropout) if dropout is not None else z
        return z


In [15]:
def tensor_log_loss(y, y_hat, eps=1e-15):
    """
    Calculates log loss between two tensors.

    Args:
        y: Label tensor.
        y_hat: Prediction tensor

    Returns:
        Log loss. 0-dimensional tensor.
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.minimum(tf.maximum(y_hat, eps), 1.0 - eps)
    log_loss = -tf.reduce_mean(y*tf.log(y_hat) + (1.0 - y)*tf.log(1.0 - y_hat))
    return log_loss


In [20]:
 with tf.Graph().as_default() as graph:
        
    user_id = tf.placeholder(tf.int32, [None])
    master_aisle_id = tf.placeholder(tf.int32, [None])
    master_department_id = tf.placeholder(tf.int32, [None])
    history_length = tf.placeholder(tf.int32, [None])

    IsInOrder_history = tf.placeholder(tf.int32, [None, N]) #
    order_dow_history = tf.placeholder(tf.int32, [None, N])
    order_hour_of_day_history = tf.placeholder(tf.int32, [None, N])
    days_since_prior_order_history = tf.placeholder(tf.int32, [None, N])
    OrderSize_history = tf.placeholder(tf.int32, [None, N]) #
    IndexInOrder_history = tf.placeholder(tf.int32, [None, N]) #
    order_number_history = tf.placeholder(tf.int32, [None, N])
    NumProductsFromDep_history = tf.placeholder(tf.int32, [None, N]) #
    NextInOrder_history = tf.placeholder(tf.int32, [None, N])
    
    label = tf.placeholder(tf.int32, [None])

    
    #ProductID1_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID2_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID3_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID4_history_a = tf.placeholder(tf.int32, [None, N])   
    #ProductID5_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID6_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID7_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID8_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID9_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID11_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID10_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID12_history_a = tf.placeholder(tf.int32, [None, N])   
    #ProductID13_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID14_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID15_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID16_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID17_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID18_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID19_history_a = tf.placeholder(tf.int32, [None, N])
    #ProductID20_history_a = tf.placeholder(tf.int32, [None, N])
    
    #ProductNameEmbedding_history = tf.placeholder(tf.float32, [None, N, 50])

    ProductID1_history = tf.placeholder(tf.int32, [None, N])
    ProductID2_history = tf.placeholder(tf.int32, [None, N])
    ProductID3_history = tf.placeholder(tf.int32, [None, N])
    ProductID4_history = tf.placeholder(tf.int32, [None, N])   
    ProductID5_history = tf.placeholder(tf.int32, [None, N])
    ProductID6_history = tf.placeholder(tf.int32, [None, N])
    ProductID7_history = tf.placeholder(tf.int32, [None, N])
    ProductID8_history = tf.placeholder(tf.int32, [None, N])
    ProductID9_history = tf.placeholder(tf.int32, [None, N])
    ProductID11_history = tf.placeholder(tf.int32, [None, N])
    ProductID10_history = tf.placeholder(tf.int32, [None, N])
    ProductID12_history = tf.placeholder(tf.int32, [None, N])   
    ProductID13_history = tf.placeholder(tf.int32, [None, N])
    ProductID14_history = tf.placeholder(tf.int32, [None, N])
    ProductID15_history = tf.placeholder(tf.int32, [None, N])
    ProductID16_history = tf.placeholder(tf.int32, [None, N])
    ProductID17_history = tf.placeholder(tf.int32, [None, N])
    ProductID18_history = tf.placeholder(tf.int32, [None, N])
    ProductID19_history = tf.placeholder(tf.int32, [None, N])
    ProductID20_history = tf.placeholder(tf.int32, [None, N])
  
    keep_prob = tf.placeholder(tf.float32)
    is_training = tf.placeholder(tf.bool)
    
    # aisle data
    aisle_embeddings = tf.get_variable(
        name='aisle_embeddings',
        shape=[135, 50],
        dtype=tf.float32
    )
  
    # department data
    dept_embeddings = tf.get_variable(
        name='dept_embeddings',
        shape=[22, 10],
        dtype=tf.float32
    )
  
    x_aisle = tf.concat([
        tf.nn.embedding_lookup(aisle_embeddings, master_aisle_id),
        tf.nn.embedding_lookup(dept_embeddings, master_department_id),
    ], axis=1)
    x_aisle = tf.tile(tf.expand_dims(x_aisle, 1), (1, N, 1))

    # user data
    user_embeddings = tf.get_variable(
        name='user_embeddings',
        shape=[207000, min(lstm_size,50)],
        dtype=tf.float32
    )
    x_user = tf.nn.embedding_lookup(user_embeddings, user_id)
    x_user = tf.tile(tf.expand_dims(x_user, 1), (1, N, 1))

    # product data
    product_embeddings = tf.get_variable(
        name='product_embeddings',
        shape=[50000, 50],
        dtype=tf.float32
    )
       
    #x_product_1a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID1_history_a))
    #x_product_2a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID2_history_a))
    #x_product_3a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID3_history_a))
    #x_product_4a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID4_history_a))
    #x_product_5a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID5_history_a))
    #x_product_6a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID6_history_a))
    #x_product_7a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID7_history_a))
    #x_product_8a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID8_history_a))
    #x_product_9a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID9_history_a))
    #x_product_10a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID10_history_a))
    #x_product_11a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID11_history_a))
    #x_product_12a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID12_history_a))
    #x_product_13a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID13_history_a))
    #x_product_14a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID14_history_a))
    #x_product_15a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID15_history_a))
    #x_product_16a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID16_history_a))
    #x_product_17a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID17_history_a))
    #x_product_18a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID18_history_a))
    #x_product_19a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID19_history_a))
    #x_product_20a = tf.nn.embedding_lookup(aisle_embeddings, (ProductID20_history_a))
    
    x_product_1 = tf.nn.embedding_lookup(product_embeddings, (ProductID1_history))
    x_product_2 = tf.nn.embedding_lookup(product_embeddings, (ProductID2_history))
    x_product_3 = tf.nn.embedding_lookup(product_embeddings, (ProductID3_history))
    x_product_4 = tf.nn.embedding_lookup(product_embeddings, (ProductID4_history))
    x_product_5 = tf.nn.embedding_lookup(product_embeddings, (ProductID5_history))
    x_product_6 = tf.nn.embedding_lookup(product_embeddings, (ProductID6_history))
    x_product_7 = tf.nn.embedding_lookup(product_embeddings, (ProductID7_history))
    x_product_8 = tf.nn.embedding_lookup(product_embeddings, (ProductID8_history))
    x_product_9 = tf.nn.embedding_lookup(product_embeddings, (ProductID9_history))
    x_product_10 = tf.nn.embedding_lookup(product_embeddings, (ProductID10_history))
    x_product_11 = tf.nn.embedding_lookup(product_embeddings, (ProductID11_history))
    x_product_12 = tf.nn.embedding_lookup(product_embeddings, (ProductID12_history))
    x_product_13 = tf.nn.embedding_lookup(product_embeddings, (ProductID13_history))
    x_product_14 = tf.nn.embedding_lookup(product_embeddings, (ProductID14_history))
    x_product_15 = tf.nn.embedding_lookup(product_embeddings, (ProductID15_history))
    x_product_16 = tf.nn.embedding_lookup(product_embeddings, (ProductID16_history))
    x_product_17 = tf.nn.embedding_lookup(product_embeddings, (ProductID17_history))
    x_product_18 = tf.nn.embedding_lookup(product_embeddings, (ProductID18_history))
    x_product_19 = tf.nn.embedding_lookup(product_embeddings, (ProductID19_history))
    x_product_20 = tf.nn.embedding_lookup(product_embeddings, (ProductID20_history))
    
   
    # sequence data
    IsInOrder_history_oh = tf.one_hot(IsInOrder_history, 2)
    order_dow_history_oh = tf.one_hot(order_dow_history, 8)
    order_hour_of_day_history_oh = tf.one_hot(order_hour_of_day_history, 25)
    days_since_prior_order_history_oh = tf.one_hot(days_since_prior_order_history, 31)
    OrderSize_history_oh = tf.one_hot(OrderSize_history, 60)
    IndexInOrder_history_oh = tf.one_hot(OrderSize_history, 60)
    order_number_history_oh = tf.one_hot(order_number_history, 101)
    NumProductsFromDep_history_oh = tf.one_hot(NumProductsFromDep_history, 50)
    
    order_dow_history_scalar = tf.expand_dims(tf.cast(order_dow_history, tf.float32) / 8.0, 2)
    order_hour_of_day_history_scalar = tf.expand_dims(tf.cast(order_hour_of_day_history, tf.float32) / 25.0, 2)
    days_since_prior_order_history_scalar = tf.expand_dims(tf.cast(days_since_prior_order_history, tf.float32) / 31.0, 2)
    OrderSize_history_scalar = tf.expand_dims(tf.cast(OrderSize_history, tf.float32) / 60.0, 2)
    IndexInOrder_history_scalar = tf.expand_dims(tf.cast(IndexInOrder_history, tf.float32) / 60.0, 2)
    order_number_history_scalar = tf.expand_dims(tf.cast(order_number_history, tf.float32) / 100.0, 2)
    NumProductsFromDep_history_scalar = tf.expand_dims(tf.cast(NumProductsFromDep_history, tf.float32) / 50.0, 2)



    x_history = tf.concat([IsInOrder_history_oh,order_dow_history_oh,order_hour_of_day_history_oh,days_since_prior_order_history_oh,\
        OrderSize_history_oh,IndexInOrder_history_oh,NumProductsFromDep_history_oh,order_number_history_oh,order_dow_history_scalar,\
        order_hour_of_day_history_scalar,days_since_prior_order_history_scalar,OrderSize_history_scalar,IndexInOrder_history_scalar,\
        order_number_history_scalar,NumProductsFromDep_history_scalar,x_product_1,x_product_2,x_product_3,\
        x_product_4,x_product_5,x_product_6,x_product_7,x_product_8,x_product_9\
        ,x_product_10,x_product_11,x_product_12,x_product_13,\
        x_product_14,x_product_15,x_product_16,x_product_17,x_product_18,\
        x_product_19,x_product_20#,x_product_1a,x_product_2a,x_product_3a,\
        #x_product_4a,x_product_5a,x_product_6a,x_product_7a,x_product_8a,x_product_9a\
        #,x_product_10a,x_product_11a,x_product_12a,x_product_13a,\
        #x_product_14a,x_product_15a,x_product_16a,x_product_17a,x_product_18a,\
        #x_product_19a,x_product_20a], axis=2) #ProductNameEmbedding_history
        ], axis=2)
    
    x = tf.concat([x_history, x_aisle, x_user], axis=2)
    #h = lstm_layer(x, history_length, lstm_size, scope='lstm1')

    if WaveNet:
        c = wavenet(x, dilations, filter_widths, skip_channels, residual_channels)
        h = tf.concat([c , h], axis=2)
    else:
        h = lstm_layer(x, history_length, lstm_size, scope='lstm1')
        h = tf.concat([h , x], axis=2)
    

    h_final = time_distributed_dense_layer(h, 50 , activation=tf.nn.relu, scope='dense1') 
    #y_hat = tf.squeeze(time_distributed_dense_layer(h_final, 1, activation=tf.nn.sigmoid, scope='dense3'), 2)

    n_components = 1
    params = time_distributed_dense_layer(h_final, n_components*2, scope='dense-2', activation=None)
    ps, mixing_coefs = tf.split(params, 2, axis=2)

    # this is implemented incorrectly, but it still helped...
    mixing_coefs = tf.nn.softmax(mixing_coefs - tf.reduce_min(mixing_coefs, 2, keep_dims=True))
    ps = tf.nn.sigmoid(ps)

    #loss = sequence_log_loss(NextInOrder_history, preds, history_length, N)
    labels = tf.tile(tf.expand_dims(NextInOrder_history, 2), (1, 1, n_components))
    losses = tf.reduce_sum(mixing_coefs*tensor_log_loss(labels, ps), axis=2)
    sequence_mask = tf.cast(tf.sequence_mask(history_length, maxlen=N), tf.float32)
    loss = tf.reduce_sum(losses*sequence_mask) / tf.cast(tf.reduce_sum(history_length), tf.float32)

    
    final_temporal_idx = tf.stack([tf.range(tf.shape(history_length)[0]), history_length - 1], axis=1)
    final_states = tf.gather_nd(h_final, final_temporal_idx)
    #final_predictions = tf.gather_nd(y_hat, final_temporal_idx)
    final_predictions = tf.gather_nd(ps, final_temporal_idx)
    ground_truth = tf.gather_nd(NextInOrder_history, final_temporal_idx)
    
    prediction_tensors = {
        'user_ids': user_id,
        'aisle_ids': master_aisle_id,
        'final_states': final_states,
        'predictions': final_predictions,
        'ground_truth': ground_truth,
        'history_length':history_length
    }
    #preds = y_hat
    preds = ps
        

    global_step = tf.Variable(0, trainable=False)
    learning_rate_var = tf.Variable(0.0, trainable=False)

    if regularization_constant != 0:
        l2_norm = tf.reduce_sum([tf.sqrt(tf.reduce_sum(tf.square(param))) for param in tf.trainable_variables()])
        loss = loss + regularization_constant*l2_norm

    if opt == 'adam':
        optimizer =  tf.train.AdamOptimizer(learning_rate_var)
    elif opt == 'gd':
        optimizer =  tf.train.GradientDescentOptimizer(learning_rate_var)
    elif opt == 'rms':
        optimizer =  tf.train.RMSPropOptimizer(learning_rate_var, decay=0.95, momentum=0.9)
    else:
        assert False, 'optimizer must be adam, gd, or rms'
            
    grads = optimizer.compute_gradients(loss)
    clipped = [(tf.clip_by_value(g, -grad_clip, grad_clip), v_) for g, v_ in grads]

    step = optimizer.apply_gradients(clipped, global_step=global_step)

    print('trainable parameters:')
    print([(var.name, shape(var)) for var in tf.trainable_variables()])

    print('trainable parameter count:')
    print(str(np.sum(np.prod(shape(var)) for var in tf.trainable_variables())))

    saver = tf.train.Saver(max_to_keep=1)
    
    init = tf.global_variables_initializer()
    
session = tf.Session(graph=graph)
print('built graph')

trainable parameters:
[('aisle_embeddings:0', [135, 50]), ('dept_embeddings:0', [22, 10]), ('user_embeddings:0', [207000, 50]), ('product_embeddings:0', [50000, 50]), ('lstm1/rnn/lstm_cell/kernel:0', [1754, 1200]), ('lstm1/rnn/lstm_cell/bias:0', [1200]), ('dense1/weights:0', [1754, 50]), ('dense1/biases:0', [50]), ('dense-2/weights:0', [50, 2]), ('dense-2/biases:0', [2])]
trainable parameter count:
15050822
built graph


In [21]:
with session.as_default():
    debug = False

    if warm_start_init_step:
        restore(warm_start_init_step)
        step_number = warm_start_init_step
    else:
        session.run(init)
        step_number = 0

    best_validation_loss, best_validation_tstep = float('inf'), 0
    restarts = 0
    
    train_loss_history = deque(maxlen=loss_averaging_window)
    val_loss_history = deque(maxlen=loss_averaging_window)
    val_seq_loss_history = deque(maxlen=loss_averaging_window)
    
    while step_number < num_training_steps:

        next_batch =  next(train.next_batch(batch_size))
                    
        # train step
        train_feed_dict = next_batch

        # Use n-2 for training so that n-1 can be used for validation

        if debug:
            print(next_batch[globals()['history_length']])
        train_feed_dict.update({globals()['history_length']:train_feed_dict[globals()['history_length']]-1})
        
        if debug:
            print(train_feed_dict[globals()['history_length']])

        sequence_lengths = train_feed_dict[globals()['history_length']]
        if debug:
            print(sequence_lengths)
        
        for i in range(batch_size):
        
        # Remove the input data for n-1 step for training to avoid leakage

            temp_array = train_feed_dict[globals()['NextInOrder_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['NextInOrder_history']:temp_array})
            
            temp_array = train_feed_dict[globals()['order_number_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['order_number_history']:temp_array})

            temp_array = train_feed_dict[globals()['order_hour_of_day_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['order_hour_of_day_history']:temp_array})

            temp_array = train_feed_dict[globals()['order_dow_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['order_dow_history']:temp_array})

            temp_array = train_feed_dict[globals()['days_since_prior_order_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['days_since_prior_order_history']:temp_array})
                       
            temp_array = train_feed_dict[globals()['IsInOrder_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['IsInOrder_history']:temp_array})

            temp_array = train_feed_dict[globals()['OrderSize_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['OrderSize_history']:temp_array})

            temp_array = train_feed_dict[globals()['IndexInOrder_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['IndexInOrder_history']:temp_array})

            temp_array = train_feed_dict[globals()['NumProductsFromDep_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['NumProductsFromDep_history']:temp_array})

            if False:
                temp_array = train_feed_dict[globals()['ProductID1_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID1_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID2_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID2_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID3_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID3_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID4_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID4_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID5_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID5_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID6_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID6_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID7_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID7_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID8_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID8_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID9_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID9_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID10_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID10_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID11_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID11_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID12_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID12_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID13_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID13_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID14_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID14_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID15_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID15_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID16_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID16_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID17_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID17_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID18_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID18_history_a']:temp_array})      

                temp_array = train_feed_dict[globals()['ProductID19_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID19_history_a']:temp_array})

                temp_array = train_feed_dict[globals()['ProductID20_history_a']]
                temp_array[i,sequence_lengths[i]]=0
                train_feed_dict.update({globals()['ProductID20_history_a']:temp_array})

                #temp_array = train_feed_dict[globals()['ProductNameEmbedding_history']]
                #temp_array[i,sequence_lengths[i]]=np.zeros(shape=[50],dtype=np.float16)
                #train_feed_dict.update({globals()['ProductNameEmbedding_history']:temp_array})
            
            temp_array = train_feed_dict[globals()['ProductID1_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID1_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID2_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID2_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID3_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID3_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID4_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID4_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID5_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID5_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID6_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID6_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID7_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID7_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID8_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID8_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID9_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID9_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID10_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID10_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID11_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID11_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID12_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID12_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID13_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID13_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID14_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID14_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID15_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID15_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID16_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID16_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID17_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID17_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID18_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID18_history']:temp_array})      

            temp_array = train_feed_dict[globals()['ProductID19_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID19_history']:temp_array})

            temp_array = train_feed_dict[globals()['ProductID20_history']]
            temp_array[i,sequence_lengths[i]]=0
            train_feed_dict.update({globals()['ProductID20_history']:temp_array})

            
            
        train_feed_dict.update({learning_rate_var: learning_rate})
        train_feed_dict.update({keep_prob: keep_prob_scalar})
        train_feed_dict.update({is_training: True})
                
        debug_dict = {tensor_name: [] for tensor_name in prediction_tensors} #changed
        tensor_names, tf_tensors = zip(*prediction_tensors.items()) #changed

        train_loss,_,np_tensors = session.run(   #changed
            fetches=[loss,step,tf_tensors], #changed
            feed_dict=train_feed_dict
        )
        
        for tensor_name, tensor in zip(tensor_names, np_tensors): #changed
            debug_dict[tensor_name].append(tensor)#changed 
        
        if debug:
            print(debug_dict['history_length'])#changed
        
        train_loss_history.append(train_loss)
        
        #BEGIN
        prediction_dict = {tensor_name: [] for tensor_name in prediction_tensors}

        # test evaluation
        
        val_feed_dict = next_batch
        val_feed_dict.update({learning_rate_var: learning_rate})
        val_feed_dict.update({keep_prob: keep_prob_scalar})
        val_feed_dict.update({is_training: False})

        if debug:
            print(val_feed_dict[globals()['history_length']]+1)
        
        tensor_names, tf_tensors = zip(*prediction_tensors.items())
        val_seq_loss,np_tensors = session.run(
            fetches=[loss,tf_tensors],
            feed_dict=val_feed_dict
        )

        for tensor_name, tensor in zip(tensor_names, np_tensors):
            prediction_dict[tensor_name].append(tensor)
      
        val_loss = log_loss(prediction_dict['ground_truth'][0],prediction_dict['predictions'][0])
        #END
        val_loss_history.append(val_loss)
        val_seq_loss_history.append(val_seq_loss)
        
        if debug:
            #print(train_loss, val_loss)
            #print(prediction_dict['ground_truth'])
            print(prediction_dict['history_length'])
            break

        if step_number % log_interval == 0:
            avg_train_loss = sum(train_loss_history)/len(train_loss_history)
            avg_val_loss = sum(val_loss_history)/len(val_loss_history)
            avg_val_seq_loss = sum(val_seq_loss_history)/len(val_seq_loss_history)
            metric_log = (
                "[[step {:>8}]]     "
                "[[train]]     loss: {:<12}     "
                "[[val]]     loss: {:<12}     "
                "[[val-seq]]     loss: {:<12}     "
            ).format(step_number, round(avg_train_loss, 8), round(avg_val_loss, 8), round(avg_val_seq_loss, 8))
            print(metric_log)
            
            if avg_val_loss < best_validation_loss:
                best_validation_loss = avg_val_loss
                best_validation_tstep = step_number
                if step_number > min_steps_to_checkpoint:
                    model_path = os.path.join(checkpoint_dir, 'model{}.ckpt'.format(best_validation_tstep))
                    saver.save(session, model_path, global_step=best_validation_tstep) 
                    
            if step_number - best_validation_tstep > early_stopping_steps:

                if num_restarts is None or restarts >= num_restarts:
                    print('best validation loss of {} at training step {}'.format(
                        best_validation_loss, best_validation_tstep))
                    print('early stopping - ending training.')
                    break

                if restarts < num_restarts:
                    model_path = os.path.join(checkpoint_dir, 'model{}.ckpt-{}'.format(best_validation_tstep,best_validation_tstep))
                    saver.restore(session, model_path)
                    print('halving learning rate')
                    learning_rate /= 2.0
                    early_stopping_steps /= 2
                    step_number = best_validation_tstep
                    restarts += 1

        step_number += 1

    if step_number <= min_steps_to_checkpoint:
        best_validation_tstep = step_number
        model_path = os.path.join(checkpoint_dir, 'model{}.ckpt'.format(best_validation_tstep))
        saver.save(session, model_path, global_step=best_validation_tstep) 
                    
    print('num_training_steps reached - ending training')

[[step        0]]     [[train]]     loss: 0.68232197       [[val]]     loss: 0.68599959       [[val-seq]]     loss: 0.6716857        
[[step     1000]]     [[train]]     loss: 0.10434882       [[val]]     loss: 0.50269874       [[val-seq]]     loss: 0.10351389       
[[step     2000]]     [[train]]     loss: 0.08133827       [[val]]     loss: 0.48954841       [[val-seq]]     loss: 0.08100628       
[[step     3000]]     [[train]]     loss: 0.08045529       [[val]]     loss: 0.47539102       [[val-seq]]     loss: 0.0800922        
[[step     4000]]     [[train]]     loss: 0.07994609       [[val]]     loss: 0.46771655       [[val-seq]]     loss: 0.07956188       
[[step     5000]]     [[train]]     loss: 0.07979196       [[val]]     loss: 0.46430372       [[val-seq]]     loss: 0.07939229       
[[step     6000]]     [[train]]     loss: 0.07997105       [[val]]     loss: 0.46312255       [[val-seq]]     loss: 0.07956305       
[[step     7000]]     [[train]]     loss: 0.07959625       [[v

[[step    62000]]     [[train]]     loss: 0.07537154       [[val]]     loss: 0.413019         [[val-seq]]     loss: 0.07490084       
[[step    63000]]     [[train]]     loss: 0.0756379        [[val]]     loss: 0.41322129       [[val-seq]]     loss: 0.07516719       
[[step    64000]]     [[train]]     loss: 0.07536603       [[val]]     loss: 0.41245326       [[val-seq]]     loss: 0.07488664       
[[step    65000]]     [[train]]     loss: 0.07526311       [[val]]     loss: 0.41284661       [[val-seq]]     loss: 0.07478981       
[[step    66000]]     [[train]]     loss: 0.07566459       [[val]]     loss: 0.41498711       [[val-seq]]     loss: 0.07519048       
[[step    67000]]     [[train]]     loss: 0.07600604       [[val]]     loss: 0.4102098        [[val-seq]]     loss: 0.07552564       
[[step    68000]]     [[train]]     loss: 0.07536942       [[val]]     loss: 0.41124126       [[val-seq]]     loss: 0.0748886        
[[step    69000]]     [[train]]     loss: 0.0755214        [[v

[[step   125000]]     [[train]]     loss: 0.07395663       [[val]]     loss: 0.39155695       [[val-seq]]     loss: 0.07338876       
[[step   126000]]     [[train]]     loss: 0.07336153       [[val]]     loss: 0.38946542       [[val-seq]]     loss: 0.07279541       
[[step   127000]]     [[train]]     loss: 0.07415682       [[val]]     loss: 0.39117238       [[val-seq]]     loss: 0.07358691       
[[step   128000]]     [[train]]     loss: 0.07399205       [[val]]     loss: 0.3920305        [[val-seq]]     loss: 0.07341879       
[[step   129000]]     [[train]]     loss: 0.07353377       [[val]]     loss: 0.38683516       [[val-seq]]     loss: 0.07296143       
[[step   130000]]     [[train]]     loss: 0.07340056       [[val]]     loss: 0.38795441       [[val-seq]]     loss: 0.07282348       
[[step   131000]]     [[train]]     loss: 0.0738987        [[val]]     loss: 0.38846179       [[val-seq]]     loss: 0.07331807       
[[step   132000]]     [[train]]     loss: 0.07398581       [[v

[[step   188000]]     [[train]]     loss: 0.07206294       [[val]]     loss: 0.35971982       [[val-seq]]     loss: 0.07134641       
[[step   189000]]     [[train]]     loss: 0.07155173       [[val]]     loss: 0.3600603        [[val-seq]]     loss: 0.07084775       
[[step   190000]]     [[train]]     loss: 0.07131578       [[val]]     loss: 0.3594293        [[val-seq]]     loss: 0.07061077       


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1694: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1694: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[[step   191000]]     [[train]]     loss: nan              [[val]]     loss: nan              [[val-seq]]     loss: nan              
[[step   192000]]     [[train]]     loss: 0.07236744       [[val]]     loss: 0.3774675        [[val-seq]]     loss: 0.07188709       
[[step   193000]]     [[train]]     loss: 0.07334141       [[val]]     loss: 0.38646228       [[val-seq]]     loss: 0.07286074       
[[step   194000]]     [[train]]     loss: 0.07419034       [[val]]     loss: 0.39522999       [[val-seq]]     loss: 0.07371237       
[[step   195000]]     [[train]]     loss: 0.07410553       [[val]]     loss: 0.39429719       [[val-seq]]     loss: 0.07364827       
[[step   196000]]     [[train]]     loss: 0.07411413       [[val]]     loss: 0.39654429       [[val-seq]]     loss: 0.07366943       
[[step   197000]]     [[train]]     loss: 0.07374326       [[val]]     loss: 0.3939028        [[val-seq]]     loss: 0.07331389       
[[step   198000]]     [[train]]     loss: 0.07360201       [[v

[[step   253000]]     [[train]]     loss: 0.07259782       [[val]]     loss: 0.37892899       [[val-seq]]     loss: 0.07216712       
[[step   254000]]     [[train]]     loss: 0.07272825       [[val]]     loss: 0.38322921       [[val-seq]]     loss: 0.07230866       
[[step   255000]]     [[train]]     loss: 0.07244787       [[val]]     loss: 0.38210882       [[val-seq]]     loss: 0.07203892       
[[step   256000]]     [[train]]     loss: 0.07221163       [[val]]     loss: 0.38095781       [[val-seq]]     loss: 0.07180948       
[[step   257000]]     [[train]]     loss: nan              [[val]]     loss: nan              [[val-seq]]     loss: nan              
[[step   258000]]     [[train]]     loss: 0.0730997        [[val]]     loss: 0.38547002       [[val-seq]]     loss: 0.0726903        
[[step   259000]]     [[train]]     loss: 0.07265147       [[val]]     loss: 0.3804719        [[val-seq]]     loss: 0.07225596       
[[step   260000]]     [[train]]     loss: 0.07274516       [[v

[[step   315000]]     [[train]]     loss: 0.08707934       [[val]]     loss: 0.49752996       [[val-seq]]     loss: 0.08662464       
[[step   316000]]     [[train]]     loss: 0.08562909       [[val]]     loss: 0.4914991        [[val-seq]]     loss: 0.08518977       
[[step   317000]]     [[train]]     loss: 0.08444945       [[val]]     loss: 0.4823785        [[val-seq]]     loss: 0.08402412       
[[step   318000]]     [[train]]     loss: 0.08262073       [[val]]     loss: 0.47252082       [[val-seq]]     loss: 0.08222297       
[[step   319000]]     [[train]]     loss: 0.08089941       [[val]]     loss: 0.46293309       [[val-seq]]     loss: 0.08052511       
[[step   320000]]     [[train]]     loss: 0.07987365       [[val]]     loss: 0.45713147       [[val-seq]]     loss: 0.0795128        
[[step   321000]]     [[train]]     loss: 0.07835572       [[val]]     loss: 0.44580176       [[val-seq]]     loss: 0.07801548       
[[step   322000]]     [[train]]     loss: 0.07720789       [[v

[[step   377000]]     [[train]]     loss: 0.07172615       [[val]]     loss: 0.37363766       [[val-seq]]     loss: 0.0712679        
[[step   378000]]     [[train]]     loss: 0.0717372        [[val]]     loss: 0.37562885       [[val-seq]]     loss: 0.07129104       
[[step   379000]]     [[train]]     loss: 0.07162637       [[val]]     loss: 0.37695455       [[val-seq]]     loss: 0.0711972        
[[step   380000]]     [[train]]     loss: 0.07164965       [[val]]     loss: 0.37743896       [[val-seq]]     loss: 0.07123956       
[[step   381000]]     [[train]]     loss: 0.07157105       [[val]]     loss: 0.37595168       [[val-seq]]     loss: 0.07117276       
[[step   382000]]     [[train]]     loss: 0.0722569        [[val]]     loss: 0.37887946       [[val-seq]]     loss: 0.07187329       
[[step   383000]]     [[train]]     loss: 0.07221874       [[val]]     loss: 0.38186655       [[val-seq]]     loss: 0.07184636       
[[step   384000]]     [[train]]     loss: 0.07266074       [[v

KeyboardInterrupt: 

In [22]:
best_validation_loss, best_validation_tstep

(0.3463681830049152, 334000)

In [23]:
model_path = os.path.join(checkpoint_dir, 'model3463{}.ckpt'.format(best_validation_tstep))
model_path
saver.save(session, model_path, global_step=best_validation_tstep) 

'C:\\Users\\chapanda\\data\\np\\checkpoints\\model3463334000.ckpt-334000'

# Log


N = 100
debug = False
batch_size = 128
num_training_steps = 300000
learning_rate = 0.001
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 35000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 5000
log_interval = 1000
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = True
dropout = 0

dilations=[2**i for i in range(6)]
#dilations=dilations+dilations
filter_widths=[2]*6
#filter_widths=filter_widths+filter_widths
skip_channels=64
residual_channels=96

trainable parameters:
[('aisle_embeddings:0', [135, 50]), ('dept_embeddings:0', [22, 10]), ('user_embeddings:0', [207000, 50]), ('lstm1/rnn/lstm_cell/kernel:0', [1754, 1200]), ('lstm1/rnn/lstm_cell/bias:0', [1200]), ('wavenet/x-proj/weights:0', [1454, 96]), ('wavenet/x-proj/biases:0', [96]), ('wavenet/cnn-0/weights:0', [2, 96, 192]), ('wavenet/cnn-0/biases:0', [192]), ('wavenet/cnn-0-proj/weights:0', [96, 160]), ('wavenet/cnn-0-proj/biases:0', [160]), ('wavenet/cnn-1/weights:0', [2, 96, 192]), ('wavenet/cnn-1/biases:0', [192]), ('wavenet/cnn-1-proj/weights:0', [96, 160]), ('wavenet/cnn-1-proj/biases:0', [160]), ('wavenet/cnn-2/weights:0', [2, 96, 192]), ('wavenet/cnn-2/biases:0', [192]), ('wavenet/cnn-2-proj/weights:0', [96, 160]), ('wavenet/cnn-2-proj/biases:0', [160]), ('wavenet/cnn-3/weights:0', [2, 96, 192]), ('wavenet/cnn-3/biases:0', [192]), ('wavenet/cnn-3-proj/weights:0', [96, 160]), ('wavenet/cnn-3-proj/biases:0', [160]), ('wavenet/cnn-4/weights:0', [2, 96, 192]), ('wavenet/cnn-4/biases:0', [192]), ('wavenet/cnn-4-proj/weights:0', [96, 160]), ('wavenet/cnn-4-proj/biases:0', [160]), ('wavenet/cnn-5/weights:0', [2, 96, 192]), ('wavenet/cnn-5/biases:0', [192]), ('wavenet/cnn-5-proj/weights:0', [96, 160]), ('wavenet/cnn-5-proj/biases:0', [160]), ('dense1/weights:0', [794, 50]), ('dense1/biases:0', [50]), ('dense2/weights:0', [50, 1]), ('dense2/biases:0', [1])]
trainable parameter count:
12957907
built graph

[[step        0]]     [[train]]     loss: 0.79283637       [[val]]     loss: 0.60624765       [[val-seq]]     loss: 0.59424144       
[[step     1000]]     [[train]]     loss: 0.38630329       [[val]]     loss: 0.43172749       [[val-seq]]     loss: 0.38367621       
[[step     2000]]     [[train]]     loss: 0.37829497       [[val]]     loss: 0.42406548       [[val-seq]]     loss: 0.37603256       
[[step     3000]]     [[train]]     loss: 0.37626992       [[val]]     loss: 0.42579533       [[val-seq]]     loss: 0.37400462       
[[step     4000]]     [[train]]     loss: 0.37561555       [[val]]     loss: 0.42661017       [[val-seq]]     loss: 0.37342759       
[[step     5000]]     [[train]]     loss: 0.37550599       [[val]]     loss: 0.42605413       [[val-seq]]     loss: 0.37332096       
[[step     6000]]     [[train]]     loss: 0.37529765       [[val]]     loss: 0.42040225       [[val-seq]]     loss: 0.37305216       
[[step     7000]]     [[train]]     loss: 0.37498146       [[val]]     loss: 0.42369202       [[val-seq]]     loss: 0.3727545        
[[step     8000]]     [[train]]     loss: 0.37255951       [[val]]     loss: 0.42120498       [[val-seq]]     loss: 0.37030773       
[[step     9000]]     [[train]]     loss: 0.37495045       [[val]]     loss: 0.42352376       [[val-seq]]     loss: 0.3726997        
[[step    10000]]     [[train]]     loss: 0.3718529        [[val]]     loss: 0.42546232       [[val-seq]]     loss: 0.36962371       
[[step    11000]]     [[train]]     loss: 0.37165166       [[val]]     loss: 0.42362015       [[val-seq]]     loss: 0.36943411       
[[step    12000]]     [[train]]     loss: 0.37223182       [[val]]     loss: 0.42399921       [[val-seq]]     loss: 0.36999945       
[[step    13000]]     [[train]]     loss: 0.37197555       [[val]]     loss: 0.42375858       [[val-seq]]     loss: 0.36979175       
[[step    14000]]     [[train]]     loss: 0.3705603        [[val]]     loss: 0.42247732       [[val-seq]]     loss: 0.36836915       
[[step    15000]]     [[train]]     loss: 0.37174754       [[val]]     loss: 0.42055816       [[val-seq]]     loss: 0.36956906       
[[step    16000]]     [[train]]     loss: 0.36986868       [[val]]     loss: 0.4280738        [[val-seq]]     loss: 0.36770084       
[[step    17000]]     [[train]]     loss: 0.36993722       [[val]]     loss: 0.42495414       [[val-seq]]     loss: 0.36776946       
[[step    18000]]     [[train]]     loss: 0.36878239       [[val]]     loss: 0.42167605       [[val-seq]]     loss: 0.3666584        
[[step    19000]]     [[train]]     loss: 0.36844754       [[val]]     loss: 0.42399498       [[val-seq]]     loss: 0.36627883       
[[step    20000]]     [[train]]     loss: 0.36800708       [[val]]     loss: 0.42623461       [[val-seq]]     loss: 0.36582198       
[[step    21000]]     [[train]]     loss: 0.36885805       [[val]]     loss: 0.42304146       [[val-seq]]     loss: 0.36672158       
[[step    22000]]     [[train]]     loss: 0.36758876       [[val]]     loss: 0.42467972       [[val-seq]]     loss: 0.36545116       
[[step    23000]]     [[train]]     loss: 0.36837649       [[val]]     loss: 0.42312657       [[val-seq]]     loss: 0.36626064       
[[step    24000]]     [[train]]     loss: 0.36790891       [[val]]     loss: 0.42422104       [[val-seq]]     loss: 0.365809         
[[step    25000]]     [[train]]     loss: 0.36705284       [[val]]     loss: 0.4248171        [[val-seq]]     loss: 0.36494182       
[[step    26000]]     [[train]]     loss: 0.36734557       [[val]]     loss: 0.42304009       [[val-seq]]     loss: 0.36525052       
[[step    27000]]     [[train]]     loss: 0.36714028       [[val]]     loss: 0.42480437       [[val-seq]]     loss: 0.36501321       
[[step    28000]]     [[train]]     loss: 0.36800031       [[val]]     loss: 0.42523621       [[val-seq]]     loss: 0.36589179       
[[step    29000]]     [[train]]     loss: 0.36751857       [[val]]     loss: 0.42673968       [[val-seq]]     loss: 0.36540843       
[[step    30000]]     [[train]]     loss: 0.36729939       [[val]]     loss: 0.42805389       [[val-seq]]     loss: 0.36519097       
[[step    31000]]     [[train]]     loss: 0.3658448        [[val]]     loss: 0.4232727        [[val-seq]]     loss: 0.36377487       
[[step    32000]]     [[train]]     loss: 0.36803557       [[val]]     loss: 0.42449223       [[val-seq]]     loss: 0.36593829       
[[step    33000]]     [[train]]     loss: 0.3669557        [[val]]     loss: 0.42459368       [[val-seq]]     loss: 0.36485877       
[[step    34000]]     [[train]]     loss: 0.36828809       [[val]]     loss: 0.42800068       [[val-seq]]     loss: 0.36620096       
[[step    35000]]     [[train]]     loss: 0.36536645       [[val]]     loss: 0.4269611        [[val-seq]]     loss: 0.36324265       
[[step    36000]]     [[train]]     loss: 0.36635399       [[val]]     loss: 0.42869828       [[val-seq]]     loss: 0.36428317       
[[step    37000]]     [[train]]     loss: 0.36469545       [[val]]     loss: 0.4243258        [[val-seq]]     loss: 0.36259389       
[[step    38000]]     [[train]]     loss: 0.36579357       [[val]]     loss: 0.42807848       [[val-seq]]     loss: 0.36372007       
[[step    39000]]     [[train]]     loss: 0.36539431       [[val]]     loss: 0.42909807       [[val-seq]]     loss: 0.36330017       
[[step    40000]]     [[train]]     loss: 0.36616532       [[val]]     loss: 0.42908202       [[val-seq]]     loss: 0.36409379       
[[step    41000]]     [[train]]     loss: 0.36390483       [[val]]     loss: 0.43058777       [[val-seq]]     loss: 0.36182936       
[[step    42000]]     [[train]]     loss: 0.36610457       [[val]]     loss: 0.43106332       [[val-seq]]     loss: 0.36402831       
best validation loss of 0.4204022465360795 at training step 6000
early stopping - ending training.
num_training_steps reached - ending training

best_validation_loss, best_validation_tstep
(0.4209459807394815, 7000)

'C:\\Users\\chapanda\\data\\np\\checkpoints\\model42097000.ckpt-7000'

path_out = 'C:\\Users\\chapanda\\data\\np\\'
#path_out = u'/cat/home/ubuntu/ikrt/mycode/np/'
N = 100
debug = False
batch_size = 128
num_training_steps = 300000
learning_rate = 0.001
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 100000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 35000
log_interval = 1000
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = False
dropout = 0

dilations=[2**i for i in range(6)]
#dilations=dilations+dilations
filter_widths=[2]*6
#filter_widths=filter_widths+filter_widths
skip_channels=64
residual_channels=96

prediction_dir = path_out + "predictions"
checkpoint_dir = path_out + "checkpoints"

if not os.path.isdir(prediction_dir):
    os.makedirs(prediction_dir)

if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)


trainable parameters:
[('aisle_embeddings:0', [135, 50]), ('dept_embeddings:0', [22, 10]), ('user_embeddings:0', [207000, 300]), ('lstm1/rnn/lstm_cell/kernel:0', [1004, 1200]), ('lstm1/rnn/lstm_cell/bias:0', [1200]), ('dense1/weights:0', [1004, 50]), ('dense1/biases:0', [50]), ('dense2/weights:0', [50, 1]), ('dense2/biases:0', [1])]
trainable parameter count:
63363271
built graph



[[step        0]]     [[train]]     loss: 0.72577935       [[val]]     loss: 0.65642903       [[val-seq]]     loss: 0.64681882       
[[step     1000]]     [[train]]     loss: 0.38667929       [[val]]     loss: 0.43641902       [[val-seq]]     loss: 0.38249544       
[[step     2000]]     [[train]]     loss: 0.37966205       [[val]]     loss: 0.42781433       [[val-seq]]     loss: 0.37516639       
[[step     3000]]     [[train]]     loss: 0.37733739       [[val]]     loss: 0.4234371        [[val-seq]]     loss: 0.37319629       
[[step     4000]]     [[train]]     loss: 0.3772767        [[val]]     loss: 0.42816841       [[val-seq]]     loss: 0.37338137       
[[step     5000]]     [[train]]     loss: 0.37526072       [[val]]     loss: 0.42579922       [[val-seq]]     loss: 0.37156339       
[[step     6000]]     [[train]]     loss: 0.37493415       [[val]]     loss: 0.42703961       [[val-seq]]     loss: 0.37122898       
[[step     7000]]     [[train]]     loss: 0.37254057       [[val]]     loss: 0.42094598       [[val-seq]]     loss: 0.36901362       
[[step     8000]]     [[train]]     loss: 0.37242638       [[val]]     loss: 0.42468829       [[val-seq]]     loss: 0.36893474       
[[step     9000]]     [[train]]     loss: 0.37198956       [[val]]     loss: 0.42421385       [[val-seq]]     loss: 0.3684996        
[[step    10000]]     [[train]]     loss: 0.37027625       [[val]]     loss: 0.42510002       [[val-seq]]     loss: 0.36685655       
[[step    11000]]     [[train]]     loss: 0.37190998       [[val]]     loss: 0.42655317       [[val-seq]]     loss: 0.36852493       
[[step    12000]]     [[train]]     loss: 0.37150671       [[val]]     loss: 0.42382367       [[val-seq]]     loss: 0.36818472       
[[step    13000]]     [[train]]     loss: 0.37044208       [[val]]     loss: 0.42254823       [[val-seq]]     loss: 0.36713486       
[[step    14000]]     [[train]]     loss: 0.36938445       [[val]]     loss: 0.42609243       [[val-seq]]     loss: 0.36610656       
[[step    15000]]     [[train]]     loss: 0.36897956       [[val]]     loss: 0.42274822       [[val-seq]]     loss: 0.36578495       
[[step    16000]]     [[train]]     loss: 0.36960152       [[val]]     loss: 0.42546837       [[val-seq]]     loss: 0.36636248       
[[step    17000]]     [[train]]     loss: 0.37039724       [[val]]     loss: 0.42589744       [[val-seq]]     loss: 0.36719882       
[[step    18000]]     [[train]]     loss: 0.36984019       [[val]]     loss: 0.42690554       [[val-seq]]     loss: 0.36665125       
[[step    19000]]     [[train]]     loss: 0.36908674       [[val]]     loss: 0.42684426       [[val-seq]]     loss: 0.36590971       
[[step    20000]]     [[train]]     loss: 0.36816688       [[val]]     loss: 0.42691754       [[val-seq]]     loss: 0.36501191       
[[step    21000]]     [[train]]     loss: 0.36886372       [[val]]     loss: 0.42378894       [[val-seq]]     loss: 0.36572918       
[[step    22000]]     [[train]]     loss: 0.36979933       [[val]]     loss: 0.42465362       [[val-seq]]     loss: 0.36664639       
[[step    23000]]     [[train]]     loss: 0.3685043        [[val]]     loss: 0.42778233       [[val-seq]]     loss: 0.36539656       
[[step    24000]]     [[train]]     loss: 0.3676134        [[val]]     loss: 0.42729685       [[val-seq]]     loss: 0.36449743       
[[step    25000]]     [[train]]     loss: 0.36701723       [[val]]     loss: 0.42717593       [[val-seq]]     loss: 0.36392438       
[[step    26000]]     [[train]]     loss: 0.36569478       [[val]]     loss: 0.42564435       [[val-seq]]     loss: 0.36264328       
[[step    27000]]     [[train]]     loss: 0.36755869       [[val]]     loss: 0.43127325       [[val-seq]]     loss: 0.36444162       
[[step    28000]]     [[train]]     loss: 0.36827562       [[val]]     loss: 0.43043082       [[val-seq]]     loss: 0.36519178       
[[step    29000]]     [[train]]     loss: 0.36570856       [[val]]     loss: 0.42953626       [[val-seq]]     loss: 0.36263079       
[[step    30000]]     [[train]]     loss: 0.36546884       [[val]]     loss: 0.43025903       [[val-seq]]     loss: 0.36240199       
[[step    31000]]     [[train]]     loss: 0.36596183       [[val]]     loss: 0.43252584       [[val-seq]]     loss: 0.3629174        
[[step    32000]]     [[train]]     loss: 0.3668334        [[val]]     loss: 0.4294476        [[val-seq]]     loss: 0.36375651       
[[step    33000]]     [[train]]     loss: 0.36659546       [[val]]     loss: 0.42983575       [[val-seq]]     loss: 0.36353752       
[[step    34000]]     [[train]]     loss: 0.36507575       [[val]]     loss: 0.43280031       [[val-seq]]     loss: 0.36204965       
[[step    35000]]     [[train]]     loss: 0.36463612       [[val]]     loss: 0.43231235       [[val-seq]]     loss: 0.36161446       
[[step    36000]]     [[train]]     loss: 0.36568933       [[val]]     loss: 0.42955957       [[val-seq]]     loss: 0.36262748       
[[step    37000]]     [[train]]     loss: 0.36467638       [[val]]     loss: 0.43380134       [[val-seq]]     loss: 0.36163716       
[[step    38000]]     [[train]]     loss: 0.36381822       [[val]]     loss: 0.43073477       [[val-seq]]     loss: 0.36078022       
[[step    39000]]     [[train]]     loss: 0.36461969       [[val]]     loss: 0.43428645       [[val-seq]]     loss: 0.36154758       
[[step    40000]]     [[train]]     loss: 0.36492028       [[val]]     loss: 0.43434485       [[val-seq]]     loss: 0.36185018       
[[step    41000]]     [[train]]     loss: 0.3641035        [[val]]     loss: 0.42990791       [[val-seq]]     loss: 0.36107208       
[[step    42000]]     [[train]]     loss: 0.36516719       [[val]]     loss: 0.43478644       [[val-seq]]     loss: 0.36213557       
[[step    43000]]     [[train]]     loss: 0.36548409       [[val]]     loss: 0.4323879        [[val-seq]]     loss: 0.36246711       
[[step    44000]]     [[train]]     loss: 0.3638115        [[val]]     loss: 0.43335473       [[val-seq]]     loss: 0.3607743        
[[step    45000]]     [[train]]     loss: 0.36366313       [[val]]     loss: 0.43759156       [[val-seq]]     loss: 0.36058636       
[[step    46000]]     [[train]]     loss: 0.36283449       [[val]]     loss: 0.43843624       [[val-seq]]     loss: 0.35955424       
[[step    47000]]     [[train]]     loss: 0.35964085       [[val]]     loss: 0.44596492       [[val-seq]]     loss: 0.35584711       
[[step    48000]]     [[train]]     loss: 0.35961326       [[val]]     loss: 0.45195176       [[val-seq]]     loss: 0.35534527       
[[step    49000]]     [[train]]     loss: 0.35746261       [[val]]     loss: 0.45494797       [[val-seq]]     loss: 0.35279568       
[[step    50000]]     [[train]]     loss: 0.35759593       [[val]]     loss: 0.46060267       [[val-seq]]     loss: 0.35289372       
[[step    51000]]     [[train]]     loss: 0.35635811       [[val]]     loss: 0.45780343       [[val-seq]]     loss: 0.35152714       
[[step    52000]]     [[train]]     loss: 0.35471936       [[val]]     loss: 0.46360691       [[val-seq]]     loss: 0.34961522       
[[step    53000]]     [[train]]     loss: 0.35465831       [[val]]     loss: 0.45896441       [[val-seq]]     loss: 0.3496902        
[[step    54000]]     [[train]]     loss: 0.35495829       [[val]]     loss: 0.46346198       [[val-seq]]     loss: 0.34972423       

path_out = 'C:\\Users\\chapanda\\data\\np\\'
#path_out = u'/cat/home/ubuntu/ikrt/mycode/np/'
N = 100
debug = False
batch_size = 128
num_training_steps = 300000
learning_rate = 0.001
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 100000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 35000
log_interval = 1000
loss_averaging_window = 1000
lstm_size = 1024
val_multiplier = 1
WaveNet = True
dropout = 0

dilations=[2**i for i in range(6)]
#dilations=dilations+dilations
filter_widths=[2]*6
#filter_widths=filter_widths+filter_widths
skip_channels=96
residual_channels=124

trainable parameters:
[('aisle_embeddings:0', [135, 50]), ('dept_embeddings:0', [22, 10]), ('user_embeddings:0', [207000, 300]), ('lstm1/rnn/lstm_cell/kernel:0', [2728, 4096]), ('lstm1/rnn/lstm_cell/bias:0', [4096]), ('wavenet/x-proj/weights:0', [1704, 124]), ('wavenet/x-proj/biases:0', [124]), ('wavenet/cnn-0/weights:0', [2, 124, 248]), ('wavenet/cnn-0/biases:0', [248]), ('wavenet/cnn-0-proj/weights:0', [124, 220]), ('wavenet/cnn-0-proj/biases:0', [220]), ('wavenet/cnn-1/weights:0', [2, 124, 248]), ('wavenet/cnn-1/biases:0', [248]), ('wavenet/cnn-1-proj/weights:0', [124, 220]), ('wavenet/cnn-1-proj/biases:0', [220]), ('wavenet/cnn-2/weights:0', [2, 124, 248]), ('wavenet/cnn-2/biases:0', [248]), ('wavenet/cnn-2-proj/weights:0', [124, 220]), ('wavenet/cnn-2-proj/biases:0', [220]), ('wavenet/cnn-3/weights:0', [2, 124, 248]), ('wavenet/cnn-3/biases:0', [248]), ('wavenet/cnn-3-proj/weights:0', [124, 220]), ('wavenet/cnn-3-proj/biases:0', [220]), ('wavenet/cnn-4/weights:0', [2, 124, 248]), ('wavenet/cnn-4/biases:0', [248]), ('wavenet/cnn-4-proj/weights:0', [124, 220]), ('wavenet/cnn-4-proj/biases:0', [220]), ('wavenet/cnn-5/weights:0', [2, 124, 248]), ('wavenet/cnn-5/biases:0', [248]), ('wavenet/cnn-5-proj/weights:0', [124, 220]), ('wavenet/cnn-5-proj/biases:0', [220]), ('dense1/weights:0', [3304, 50]), ('dense1/biases:0', [50]), ('dense2/weights:0', [50, 1]), ('dense2/biases:0', [1])]
trainable parameter count:
74197187
built graph

[[step        0]]     [[train]]     loss: 0.70939094       [[val]]     loss: 0.41096521       
[[step     1000]]     [[train]]     loss: 0.38642146       [[val]]     loss: 0.43758895       
[[step     2000]]     [[train]]     loss: 0.37751848       [[val]]     loss: 0.42608113       
[[step     3000]]     [[train]]     loss: 0.37805695       [[val]]     loss: 0.42838143       
[[step     4000]]     [[train]]     loss: 0.3765701        [[val]]     loss: 0.424398         
[[step     5000]]     [[train]]     loss: 0.37562265       [[val]]     loss: 0.42579731       
[[step     6000]]     [[train]]     loss: 0.37614318       [[val]]     loss: 0.42438959       
[[step     7000]]     [[train]]     loss: 0.37442617       [[val]]     loss: 0.42294047       
[[step     8000]]     [[train]]     loss: 0.37305277       [[val]]     loss: 0.42525389       
[[step     9000]]     [[train]]     loss: 0.37332716       [[val]]     loss: 0.42628442       
[[step    10000]]     [[train]]     loss: 0.37205556       [[val]]     loss: 0.42547762       
[[step    11000]]     [[train]]     loss: 0.37247341       [[val]]     loss: 0.42411674       
[[step    12000]]     [[train]]     loss: 0.371267         [[val]]     loss: 0.42414662       
[[step    13000]]     [[train]]     loss: 0.37237499       [[val]]     loss: 0.42269402       
[[step    14000]]     [[train]]     loss: 0.37124519       [[val]]     loss: 0.42742674       
[[step    15000]]     [[train]]     loss: 0.37024065       [[val]]     loss: 0.42706169       
[[step    16000]]     [[train]]     loss: 0.36951248       [[val]]     loss: 0.42313572       
[[step    17000]]     [[train]]     loss: 0.36920308       [[val]]     loss: 0.42702655       
[[step    18000]]     [[train]]     loss: 0.36890243       [[val]]     loss: 0.42479633       
[[step    19000]]     [[train]]     loss: 0.36838289       [[val]]     loss: 0.42760042       
[[step    20000]]     [[train]]     loss: 0.36812239       [[val]]     loss: 0.42703578       
[[step    21000]]     [[train]]     loss: 0.36878137       [[val]]     loss: 0.42523869       
[[step    22000]]     [[train]]     loss: 0.3681644        [[val]]     loss: 0.42360044       
[[step    23000]]     [[train]]     loss: 0.36752876       [[val]]     loss: 0.42814147       
[[step    24000]]     [[train]]     loss: 0.36667018       [[val]]     loss: 0.42329501       
[[step    25000]]     [[train]]     loss: 0.36729296       [[val]]     loss: 0.4259387        
[[step    26000]]     [[train]]     loss: 0.36842091       [[val]]     loss: 0.42790254       
[[step    27000]]     [[train]]     loss: 0.36803813       [[val]]     loss: 0.427691         
[[step    28000]]     [[train]]     loss: 0.36773647       [[val]]     loss: 0.43003596       
[[step    29000]]     [[train]]     loss: 0.36623077       [[val]]     loss: 0.42828908       
[[step    30000]]     [[train]]     loss: 0.36628201       [[val]]     loss: 0.42338908       
[[step    31000]]     [[train]]     loss: 0.36904426       [[val]]     loss: 0.427335         
[[step    32000]]     [[train]]     loss: 0.36725425       [[val]]     loss: 0.42920808       
[[step    33000]]     [[train]]     loss: 0.367116         [[val]]     loss: 0.42978597       
[[step    34000]]     [[train]]     loss: 0.36501117       [[val]]     loss: 0.42692595       
[[step    35000]]     [[train]]     loss: 0.36478453       [[val]]     loss: 0.42930666       
[[step    36000]]     [[train]]     loss: 0.36557198       [[val]]     loss: 0.42938079       
[[step    37000]]     [[train]]     loss: 0.36597545       [[val]]     loss: 0.43095312       
[[step    38000]]     [[train]]     loss: 0.36541395       [[val]]     loss: 0.43255321       
[[step    39000]]     [[train]]     loss: 0.36410409       [[val]]     loss: 0.43244085       
[[step    40000]]     [[train]]     loss: 0.36610092       [[val]]     loss: 0.43255489       
[[step    41000]]     [[train]]     loss: 0.36416895       [[val]]     loss: 0.43403675       
[[step    42000]]     [[train]]     loss: 0.365641         [[val]]     loss: 0.4310425        
[[step    43000]]     [[train]]     loss: 0.3640731        [[val]]     loss: 0.431747         
[[step    44000]]     [[train]]     loss: 0.36397421       [[val]]     loss: 0.43372008       
[[step    45000]]     [[train]]     loss: 0.36501272       [[val]]     loss: 0.4310498        
[[step    46000]]     [[train]]     loss: 0.36092192       [[val]]     loss: 0.43643077       
[[step    47000]]     [[train]]     loss: 0.36079551       [[val]]     loss: 0.44190613       
[[step    48000]]     [[train]]     loss: 0.35971587       [[val]]     loss: 0.4499071        
[[step    49000]]     [[train]]     loss: 0.35865844       [[val]]     loss: 0.45116917       
[[step    50000]]     [[train]]     loss: 0.35887549       [[val]]     loss: 0.45157063       
[[step    51000]]     [[train]]     loss: 0.35677896       [[val]]     loss: 0.45403695       
[[step    52000]]     [[train]]     loss: 0.35802961       [[val]]     loss: 0.45085482       
[[step    53000]]     [[train]]     loss: 0.35708232       [[val]]     loss: 0.45648349       
[[step    54000]]     [[train]]     loss: 0.35589415       [[val]]     loss: 0.45895067       
[[step    55000]]     [[train]]     loss: 0.35592514       [[val]]     loss: 0.45281368       
[[step    56000]]     [[train]]     loss: 0.3568274        [[val]]     loss: 0.45942318       
[[step    57000]]     [[train]]     loss: 0.35511323       [[val]]     loss: 0.4577161        
[[step    58000]]     [[train]]     loss: 0.35569286       [[val]]     loss: 0.46056          
[[step    59000]]     [[train]]     loss: 0.35638797       [[val]]     loss: 0.4578978        
[[step    60000]]     [[train]]     loss: 0.35509676       [[val]]     loss: 0.46222879       
[[step    61000]]     [[train]]     loss: 0.35391866       [[val]]     loss: 0.45974356       
[[step    62000]]     [[train]]     loss: 0.35398123       [[val]]     loss: 0.46046257       
[[step    63000]]     [[train]]     loss: 0.35339813       [[val]]     loss: 0.46283644       
[[step    64000]]     [[train]]     loss: 0.35366576       [[val]]     loss: 0.46394283       
[[step    65000]]     [[train]]     loss: 0.3531314        [[val]]     loss: 0.46213683       
[[step    66000]]     [[train]]     loss: 0.35327068       [[val]]     loss: 0.46291079       


#path_out = 'C:\\Users\\chapanda\\data\\'
path_out = u'/cat/home/ubuntu/ikrt/mycode/np/'
debug = False
batch_size = 512
num_training_steps = 200000
learning_rate = 0.001
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 100000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 100000
log_interval = 1000
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = True
dropout = 0

dilations=[2**i for i in range(6)]
#dilations=dilations+dilations
filter_widths=[2]*6
#filter_widths=filter_widths+filter_widths
skip_channels=124
residual_channels=256

prediction_dir = path_out + "predictions"
checkpoint_dir = path_out + "checkpoints"

if not os.path.isdir(prediction_dir):
    os.makedirs(prediction_dir)

if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)


trainable parameters:
[(u'aisle_embeddings:0', [135, 10]), (u'dept_embeddings:0', [22, 5]), (u'user_embeddings:0', [207000, 10]), (u'wavenet/x-proj/weights:0', [569, 256]), (u'wavenet/x-proj/biases:0', [256]), (u'wavenet/cnn-0/weights:0', [2, 256, 512]), (u'wavenet/cnn-0/biases:0', [512]), (u'wavenet/cnn-0-proj/weights:0', [256, 380]), (u'wavenet/cnn-0-proj/biases:0', [380]), (u'wavenet/cnn-1/weights:0', [2, 256, 512]), (u'wavenet/cnn-1/biases:0', [512]), (u'wavenet/cnn-1-proj/weights:0', [256, 380]), (u'wavenet/cnn-1-proj/biases:0', [380]), (u'wavenet/cnn-2/weights:0', [2, 256, 512]), (u'wavenet/cnn-2/biases:0', [512]), (u'wavenet/cnn-2-proj/weights:0', [256, 380]), (u'wavenet/cnn-2-proj/biases:0', [380]), (u'wavenet/cnn-3/weights:0', [2, 256, 512]), (u'wavenet/cnn-3/biases:0', [512]), (u'wavenet/cnn-3-proj/weights:0', [256, 380]), (u'wavenet/cnn-3-proj/biases:0', [380]), (u'wavenet/cnn-4/weights:0', [2, 256, 512]), (u'wavenet/cnn-4/biases:0', [512]), (u'wavenet/cnn-4-proj/weights:0', [256, 380]), (u'wavenet/cnn-4-proj/biases:0', [380]), (u'wavenet/cnn-5/weights:0', [2, 256, 512]), (u'wavenet/cnn-5/biases:0', [512]), (u'wavenet/cnn-5-proj/weights:0', [256, 380]), (u'wavenet/cnn-5-proj/biases:0', [380]), (u'dense1/weights:0', [1313, 100]), (u'dense1/biases:0', [100]), (u'dense4/weights:0', [100, 1]), (u'dense4/biases:0', [1])]
trainable parameter count:
4510777
built graph

[[step        0]]     [[train]]     loss: 0.65639955       [[val]]     loss: 0.6460228        
[[step     1000]]     [[train]]     loss: 0.39410796       [[val]]     loss: 0.39500415       
[[step     2000]]     [[train]]     loss: 0.39043517       [[val]]     loss: 0.38833185       
[[step     3000]]     [[train]]     loss: 0.38773643       [[val]]     loss: 0.38830759       
[[step     4000]]     [[train]]     loss: 0.38594667       [[val]]     loss: 0.38734226       
[[step     5000]]     [[train]]     loss: 0.38534158       [[val]]     loss: 0.38749352       
[[step     6000]]     [[train]]     loss: 0.38586397       [[val]]     loss: 0.3863855        
[[step     7000]]     [[train]]     loss: 0.38522986       [[val]]     loss: 0.38695843       
[[step     8000]]     [[train]]     loss: 0.38583578       [[val]]     loss: 0.38644603       
[[step     9000]]     [[train]]     loss: 0.38407461       [[val]]     loss: 0.3873567        
[[step    10000]]     [[train]]     loss: 0.38389422       [[val]]     loss: 0.38706803       
[[step    11000]]     [[train]]     loss: 0.38081008       [[val]]     loss: 0.38797855       
[[step    12000]]     [[train]]     loss: 0.37900639       [[val]]     loss: 0.38802069       
[[step    13000]]     [[train]]     loss: 0.37913274       [[val]]     loss: 0.38738997       
[[step    14000]]     [[train]]     loss: 0.37896445       [[val]]     loss: 0.38823297       
[[step    15000]]     [[train]]     loss: 0.37926663       [[val]]     loss: 0.38674552       
[[step    16000]]     [[train]]     loss: 0.37970057       [[val]]     loss: 0.38887942       
[[step    17000]]     [[train]]     loss: 0.37902393       [[val]]     loss: 0.388228         
[[step    18000]]     [[train]]     loss: 0.38008267       [[val]]     loss: 0.38851277       
[[step    19000]]     [[train]]     loss: 0.3773005        [[val]]     loss: 0.38779979       
[[step    20000]]     [[train]]     loss: 0.37470627       [[val]]     loss: 0.38891269       
[[step    21000]]     [[train]]     loss: 0.3753976        [[val]]     loss: 0.38834771       
[[step    22000]]     [[train]]     loss: 0.3749405        [[val]]     loss: 0.3893597        
[[step    23000]]     [[train]]     loss: 0.37535476       [[val]]     loss: 0.38784994       
[[step    24000]]     [[train]]     loss: 0.37633207       [[val]]     loss: 0.39032121       
[[step    25000]]     [[train]]     loss: 0.37576604       [[val]]     loss: 0.38850753       
[[step    26000]]     [[train]]     loss: 0.37804546       [[val]]     loss: 0.39029557       
[[step    27000]]     [[train]]     loss: 0.37477509       [[val]]     loss: 0.38876165       
[[step    28000]]     [[train]]     loss: 0.37353205       [[val]]     loss: 0.38979194       
[[step    29000]]     [[train]]     loss: 0.37330536       [[val]]     loss: 0.3893781        
[[step    30000]]     [[train]]     loss: 0.37306514       [[val]]     loss: 0.39048891       
[[step    31000]]     [[train]]     loss: 0.3734858        [[val]]     loss: 0.38949871       
[[step    32000]]     [[train]]     loss: 0.37519194       [[val]]     loss: 0.38989895       
[[step    33000]]     [[train]]     loss: 0.37359269       [[val]]     loss: 0.38998747       
[[step    34000]]     [[train]]     loss: 0.37674954       [[val]]     loss: 0.39042301       
[[step    35000]]     [[train]]     loss: 0.37249341       [[val]]     loss: 0.38951659       

Pass 1

#path_out = 'C:\\Users\\chapanda\\data\\'
path_out = u'/cat/home/ubuntu/ikrt/mycode/np/'
debug = False
batch_size = 512
num_training_steps = 200000
learning_rate = 0.0002
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 100000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 100000
log_interval = 1000
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = True
dropout = 0

dilations=[2**i for i in range(6)]
dilations=dilations+dilations
filter_widths=[2]*6
filter_widths=filter_widths+filter_widths
skip_channels=96
residual_channels=124

prediction_dir = path_out + "predictions"
checkpoint_dir = path_out + "checkpoints"

if not os.path.isdir(prediction_dir):
    os.makedirs(prediction_dir)

if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

trainable parameters:
[(u'aisle_embeddings:0', [135, 50]), (u'dept_embeddings:0', [22, 10]), (u'wavenet/x-proj/weights:0', [1404, 124]), (u'wavenet/x-proj/biases:0', [124]), (u'wavenet/cnn-0/weights:0', [2, 124, 248]), (u'wavenet/cnn-0/biases:0', [248]), (u'wavenet/cnn-0-proj/weights:0', [124, 220]), (u'wavenet/cnn-0-proj/biases:0', [220]), (u'wavenet/cnn-1/weights:0', [2, 124, 248]), (u'wavenet/cnn-1/biases:0', [248]), (u'wavenet/cnn-1-proj/weights:0', [124, 220]), (u'wavenet/cnn-1-proj/biases:0', [220]), (u'wavenet/cnn-2/weights:0', [2, 124, 248]), (u'wavenet/cnn-2/biases:0', [248]), (u'wavenet/cnn-2-proj/weights:0', [124, 220]), (u'wavenet/cnn-2-proj/biases:0', [220]), (u'wavenet/cnn-3/weights:0', [2, 124, 248]), (u'wavenet/cnn-3/biases:0', [248]), (u'wavenet/cnn-3-proj/weights:0', [124, 220]), (u'wavenet/cnn-3-proj/biases:0', [220]), (u'wavenet/cnn-4/weights:0', [2, 124, 248]), (u'wavenet/cnn-4/biases:0', [248]), (u'wavenet/cnn-4-proj/weights:0', [124, 220]), (u'wavenet/cnn-4-proj/biases:0', [220]), (u'wavenet/cnn-5/weights:0', [2, 124, 248]), (u'wavenet/cnn-5/biases:0', [248]), (u'wavenet/cnn-5-proj/weights:0', [124, 220]), (u'wavenet/cnn-5-proj/biases:0', [220]), (u'wavenet/cnn-6/weights:0', [2, 124, 248]), (u'wavenet/cnn-6/biases:0', [248]), (u'wavenet/cnn-6-proj/weights:0', [124, 220]), (u'wavenet/cnn-6-proj/biases:0', [220]), (u'wavenet/cnn-7/weights:0', [2, 124, 248]), (u'wavenet/cnn-7/biases:0', [248]), (u'wavenet/cnn-7-proj/weights:0', [124, 220]), (u'wavenet/cnn-7-proj/biases:0', [220]), (u'wavenet/cnn-8/weights:0', [2, 124, 248]), (u'wavenet/cnn-8/biases:0', [248]), (u'wavenet/cnn-8-proj/weights:0', [124, 220]), (u'wavenet/cnn-8-proj/biases:0', [220]), (u'wavenet/cnn-9/weights:0', [2, 124, 248]), (u'wavenet/cnn-9/biases:0', [248]), (u'wavenet/cnn-9-proj/weights:0', [124, 220]), (u'wavenet/cnn-9-proj/biases:0', [220]), (u'wavenet/cnn-10/weights:0', [2, 124, 248]), (u'wavenet/cnn-10/biases:0', [248]), (u'wavenet/cnn-10-proj/weights:0', [124, 220]), (u'wavenet/cnn-10-proj/biases:0', [220]), (u'wavenet/cnn-11/weights:0', [2, 124, 248]), (u'wavenet/cnn-11/biases:0', [248]), (u'wavenet/cnn-11-proj/weights:0', [124, 220]), (u'wavenet/cnn-11-proj/biases:0', [220]), (u'dense1/weights:0', [1152, 1]), (u'dense1/biases:0', [1])]
trainable parameter count:
1253367
built graph

[[step        0]]     [[train]]     loss: 0.57300597       [[val]]     loss: 0.54200459       
[[step     1000]]     [[train]]     loss: 0.4037229        [[val]]     loss: 0.40151307       
[[step     2000]]     [[train]]     loss: 0.39069892       [[val]]     loss: 0.39203746       
[[step     3000]]     [[train]]     loss: 0.38875566       [[val]]     loss: 0.38946532       
[[step     4000]]     [[train]]     loss: 0.38789024       [[val]]     loss: 0.38929309       
[[step     5000]]     [[train]]     loss: 0.38554126       [[val]]     loss: 0.38749247       
[[step     6000]]     [[train]]     loss: 0.38616442       [[val]]     loss: 0.38787958       
[[step     7000]]     [[train]]     loss: 0.38744169       [[val]]     loss: 0.38627563       
[[step     8000]]     [[train]]     loss: 0.38623234       [[val]]     loss: 0.38779106       
[[step     9000]]     [[train]]     loss: 0.38921565       [[val]]     loss: 0.38652136       
[[step    10000]]     [[train]]     loss: 0.38508331       [[val]]     loss: 0.38733072       
[[step    11000]]     [[train]]     loss: 0.38469722       [[val]]     loss: 0.38582431       
[[step    12000]]     [[train]]     loss: 0.38549126       [[val]]     loss: 0.38620556       
[[step    13000]]     [[train]]     loss: 0.38293964       [[val]]     loss: 0.38595986       
[[step    14000]]     [[train]]     loss: 0.38466007       [[val]]     loss: 0.38606849       
[[step    15000]]     [[train]]     loss: 0.38619278       [[val]]     loss: 0.38542646       
[[step    16000]]     [[train]]     loss: 0.38458682       [[val]]     loss: 0.38630333       
[[step    17000]]     [[train]]     loss: 0.38788458       [[val]]     loss: 0.3855057        
[[step    18000]]     [[train]]     loss: 0.38331083       [[val]]     loss: 0.3860205        
[[step    19000]]     [[train]]     loss: 0.38346381       [[val]]     loss: 0.38480297       
[[step    20000]]     [[train]]     loss: 0.38435932       [[val]]     loss: 0.38571543       
[[step    21000]]     [[train]]     loss: 0.38265559       [[val]]     loss: 0.38568668       
[[step    33000]]     [[train]]     loss: 0.38622362       [[val]]     loss: 0.38602395 


WaveNet ::: Training Loss :: 0.38212136 @ 11500 :: Validation Loss : 0.38483268 @ 8500

#path_out = 'C:\\Users\\chapanda\\data\\'
path_out = u'/cat/home/ubuntu/ikrt/mycode/np/'
debug = False
batch_size = 512
num_training_steps = 100000
learning_rate = 0.001
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 100000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 100000
log_interval = 500
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = True
dropout = 0

dilations=[2**i for i in range(6)]
#dilations=dilations+dilations
filter_widths=[2]*6
#filter_widths=filter_widths+filter_widths
skip_channels=96
residual_channels=124

trainable parameters:
[(u'aisle_embeddings:0', [135, 50]), (u'dept_embeddings:0', [22, 10]), (u'product_embeddings:0', [50000, 50]), (u'wavenet/x-proj/weights:0', [1404, 124]), (u'wavenet/x-proj/biases:0', [124]), (u'wavenet/cnn-0/weights:0', [2, 124, 248]), (u'wavenet/cnn-0/biases:0', [248]), (u'wavenet/cnn-0-proj/weights:0', [124, 220]), (u'wavenet/cnn-0-proj/biases:0', [220]), (u'wavenet/cnn-1/weights:0', [2, 124, 248]), (u'wavenet/cnn-1/biases:0', [248]), (u'wavenet/cnn-1-proj/weights:0', [124, 220]), (u'wavenet/cnn-1-proj/biases:0', [220]), (u'wavenet/cnn-2/weights:0', [2, 124, 248]), (u'wavenet/cnn-2/biases:0', [248]), (u'wavenet/cnn-2-proj/weights:0', [124, 220]), (u'wavenet/cnn-2-proj/biases:0', [220]), (u'wavenet/cnn-3/weights:0', [2, 124, 248]), (u'wavenet/cnn-3/biases:0', [248]), (u'wavenet/cnn-3-proj/weights:0', [124, 220]), (u'wavenet/cnn-3-proj/biases:0', [220]), (u'wavenet/cnn-4/weights:0', [2, 124, 248]), (u'wavenet/cnn-4/biases:0', [248]), (u'wavenet/cnn-4-proj/weights:0', [124, 220]), (u'wavenet/cnn-4-proj/biases:0', [220]), (u'wavenet/cnn-5/weights:0', [2, 124, 248]), (u'wavenet/cnn-5/biases:0', [248]), (u'wavenet/cnn-5-proj/weights:0', [124, 220]), (u'wavenet/cnn-5-proj/biases:0', [220]), (u'dense1/weights:0', [576, 1]), (u'dense1/biases:0', [1])]
trainable parameter count:
3217279
built graph


[[step        0]]     [[train]]     loss: 0.74986464       [[val]]     loss: 0.76376551       
[[step      500]]     [[train]]     loss: 0.39902695       [[val]]     loss: 0.39719228       
[[step     1000]]     [[train]]     loss: 0.39419912       [[val]]     loss: 0.39407802       
[[step     1500]]     [[train]]     loss: 0.38933741       [[val]]     loss: 0.39025618       
[[step     2000]]     [[train]]     loss: 0.38984062       [[val]]     loss: 0.38856346       
[[step     2500]]     [[train]]     loss: 0.39052064       [[val]]     loss: 0.38848885       
[[step     3000]]     [[train]]     loss: 0.38830204       [[val]]     loss: 0.38782649       
[[step     3500]]     [[train]]     loss: 0.38684298       [[val]]     loss: 0.38735862       
[[step     4000]]     [[train]]     loss: 0.38547107       [[val]]     loss: 0.38668211       
[[step     4500]]     [[train]]     loss: 0.38482107       [[val]]     loss: 0.38653999       
[[step     5000]]     [[train]]     loss: 0.38467534       [[val]]     loss: 0.38734918       
[[step     5500]]     [[train]]     loss: 0.38469537       [[val]]     loss: 0.38696021       
[[step     6000]]     [[train]]     loss: 0.38638392       [[val]]     loss: 0.38610079       
[[step     6500]]     [[train]]     loss: 0.38448875       [[val]]     loss: 0.38586745       
[[step     7000]]     [[train]]     loss: 0.38478305       [[val]]     loss: 0.38651106       
[[step     7500]]     [[train]]     loss: 0.38532901       [[val]]     loss: 0.38626928       
[[step     8000]]     [[train]]     loss: 0.38410955       [[val]]     loss: 0.38537588       
[[step     8500]]     [[train]]     loss: 0.38513654       [[val]]     loss: 0.38483268       
[[step     9000]]     [[train]]     loss: 0.38417677       [[val]]     loss: 0.38610536       
[[step     9500]]     [[train]]     loss: 0.38283743       [[val]]     loss: 0.38755284       
[[step    10000]]     [[train]]     loss: 0.38427715       [[val]]     loss: 0.38727841       
[[step    10500]]     [[train]]     loss: 0.38546348       [[val]]     loss: 0.38697921       
[[step    11000]]     [[train]]     loss: 0.38355408       [[val]]     loss: 0.38705426       
[[step    11500]]     [[train]]     loss: 0.38212136       [[val]]     loss: 0.3876643     

Stacked LSTM@300 + Wide WaveNet with Fixed Top ::: Training Loss :: 0.38121098 @ 11000 :: Validation Loss : 0.38525658 @ 9500

#path_out = 'C:\\Users\\chapanda\\data\\'
path_out = u'/cat/home/ubuntu/ikrt/mycode/np/'
debug = False
batch_size = 512
num_training_steps = 100000
learning_rate = 0.001
opt = 'adam'
grad_clip = 5
regularization_constant = 0.0000
warm_start_init_step = 0
early_stopping_steps = 100000
keep_prob_scalar = 1.0
enable_parameter_averaging = False
num_restarts = 0
min_steps_to_checkpoint = 100000
log_interval = 500
loss_averaging_window = 1000
lstm_size = 300
val_multiplier = 1
WaveNet = True
dropout = 0

dilations=[2**i for i in range(6)]
#dilations=dilations+dilations
filter_widths=[2]*6
#filter_widths=filter_widths+filter_widths
skip_channels=96
residual_channels=124*5

trainable parameters:
[(u'aisle_embeddings:0', [135, 50]), (u'dept_embeddings:0', [22, 10]), (u'product_embeddings:0', [50000, 50]), (u'lstm1/rnn/lstm_cell/kernel:0', [1704, 1200]), (u'lstm1/rnn/lstm_cell/bias:0', [1200]), (u'wavenet/x-proj/weights:0', [1404, 620]), (u'wavenet/x-proj/biases:0', [620]), (u'wavenet/cnn-0/weights:0', [2, 620, 1240]), (u'wavenet/cnn-0/biases:0', [1240]), (u'wavenet/cnn-0-proj/weights:0', [620, 716]), (u'wavenet/cnn-0-proj/biases:0', [716]), (u'wavenet/cnn-1/weights:0', [2, 620, 1240]), (u'wavenet/cnn-1/biases:0', [1240]), (u'wavenet/cnn-1-proj/weights:0', [620, 716]), (u'wavenet/cnn-1-proj/biases:0', [716]), (u'wavenet/cnn-2/weights:0', [2, 620, 1240]), (u'wavenet/cnn-2/biases:0', [1240]), (u'wavenet/cnn-2-proj/weights:0', [620, 716]), (u'wavenet/cnn-2-proj/biases:0', [716]), (u'wavenet/cnn-3/weights:0', [2, 620, 1240]), (u'wavenet/cnn-3/biases:0', [1240]), (u'wavenet/cnn-3-proj/weights:0', [620, 716]), (u'wavenet/cnn-3-proj/biases:0', [716]), (u'wavenet/cnn-4/weights:0', [2, 620, 1240]), (u'wavenet/cnn-4/biases:0', [1240]), (u'wavenet/cnn-4-proj/weights:0', [620, 716]), (u'wavenet/cnn-4-proj/biases:0', [716]), (u'wavenet/cnn-5/weights:0', [2, 620, 1240]), (u'wavenet/cnn-5/biases:0', [1240]), (u'wavenet/cnn-5-proj/weights:0', [620, 716]), (u'wavenet/cnn-5-proj/biases:0', [716]), (u'dense1/weights:0', [2280, 300]), (u'dense1/biases:0', [300]), (u'dense2/weights:0', [300, 200]), (u'dense2/biases:0', [200]), (u'dense3/weights:0', [200, 100]), (u'dense3/biases:0', [100]), (u'dense4/weights:0', [100, 1]), (u'dense4/biases:0', [1])]
trainable parameter count:
18089627
built graph

[[step        0]]     [[train]]     loss: 0.71439296       [[val]]     loss: 0.71579534       
[[step      500]]     [[train]]     loss: 0.39536874       [[val]]     loss: 0.39705661       
[[step     1000]]     [[train]]     loss: 0.39314378       [[val]]     loss: 0.39347515       
[[step     1500]]     [[train]]     loss: 0.38910436       [[val]]     loss: 0.39012438       
[[step     2000]]     [[train]]     loss: 0.38624256       [[val]]     loss: 0.38915163       
[[step     2500]]     [[train]]     loss: 0.38640541       [[val]]     loss: 0.38788248       
[[step     3000]]     [[train]]     loss: 0.38633301       [[val]]     loss: 0.38729562       
[[step     3500]]     [[train]]     loss: 0.38628424       [[val]]     loss: 0.38771333       
[[step     4000]]     [[train]]     loss: 0.3872626        [[val]]     loss: 0.3872555        
[[step     4500]]     [[train]]     loss: 0.3866859        [[val]]     loss: 0.38601295       
[[step     5000]]     [[train]]     loss: 0.38571788       [[val]]     loss: 0.38543805       
[[step     5500]]     [[train]]     loss: 0.38781096       [[val]]     loss: 0.38662798       
[[step     6000]]     [[train]]     loss: 0.38873766       [[val]]     loss: 0.38704623       
[[step     6500]]     [[train]]     loss: 0.3863655        [[val]]     loss: 0.38623803       
[[step     7000]]     [[train]]     loss: 0.38436648       [[val]]     loss: 0.38577226       
[[step     7500]]     [[train]]     loss: 0.38427535       [[val]]     loss: 0.38547944       
[[step     8000]]     [[train]]     loss: 0.3842475        [[val]]     loss: 0.38648178       
[[step     8500]]     [[train]]     loss: 0.38299651       [[val]]     loss: 0.38612066       
[[step     9500]]     [[train]]     loss: 0.38224757       [[val]]     loss: 0.38525658       
[[step    10000]]     [[train]]     loss: 0.38008553       [[val]]     loss: 0.38662994       
[[step    10500]]     [[train]]     loss: 0.38114074       [[val]]     loss: 0.38728664       
[[step    11000]]     [[train]]     loss: 0.38121098       [[val]]     loss: 0.3862937   



In [24]:
model_path='C:\\Users\\chapanda\\data\\np\\checkpoints\\model3463334000.ckpt-334000'
saver.restore(session, model_path)

INFO:tensorflow:Restoring parameters from C:\Users\chapanda\data\np\checkpoints\model3463334000.ckpt-334000


In [ ]:
prediction_dict = {tensor_name: [] for tensor_name in prediction_tensors}

num_val_batches = int(val.arrays[0].shape[0]/batch_size)
for i in range(num_test_batches):
    val_dict_list =  next(val.next_batch(batch_size))
        
    # test evaluation
    val_feed_dict = val_dict_list
    val_feed_dict.update({learning_rate_var: learning_rate})
    val_feed_dict.update({keep_prob: 1.0})
    val_feed_dict.update({is_training: False})

    tensor_names, tf_tensors = zip(*prediction_tensors.items())
    np_tensors = session.run(
        fetches=tf_tensors,
        feed_dict=val_feed_dict
    )
    for tensor_name, tensor in zip(tensor_names, np_tensors):
        prediction_dict[tensor_name].append(tensor)

for tensor_name, tensor in prediction_dict.items():
    np_tensor = np.concatenate(tensor, 0)
    save_file = os.path.join(prediction_dir, '{}.npy'.format(tensor_name))
    print('saving {} with shape {} to {}'.format(tensor_name, np_tensor.shape, save_file))
    np.save(save_file, np_tensor)

In [ ]:
def findLift(predics,actual,h_len, a_ids ,verbosity=0):
    
    aisles = np.unique(a_ids)
    cum_lift = 0.0
    cnt = 0

    for n in range(len(aisles)):

        predics_aisle = predics[np.where(a_ids==aisles[n])]
        actual_aisle = actual[np.where(a_ids==aisles[n])]
        h_len_aisle = h_len[np.where(a_ids==aisles[n])]

        i = np.where(predics_aisle>np.percentile(predics_aisle,90))
        j = np.where(predics_aisle>0)
        num_i = len(i[0])*1.0
        num_j = len(j[0])*1.0

        up = (np.sum(actual_aisle[i[0],h_len_aisle[i]-1])*1.0)/num_i
        down = (np.sum(actual_aisle[j[0],h_len_aisle[j]-1])*1.0)/num_j
        lift = round(up/down,3)
        if lift>0:
            cum_lift += lift
            cnt += 1

        if verbosity==1:
            print  lift , round(num_i/num_j,3), round(num_j/2000.0,2)

    return round(cum_lift/cnt,2)

In [ ]:
predics = np.load('np/predictions/predictions.npy')[0:1163264]
actual = np.load('np/NextInOrder_history_test.npy')[0:1163264,:]
h_len = np.load('np/history_length_test.npy')[0:1163264]
a_ids = np.load('np/master_aisle_id_test.npy')[0:1163264]
print predics.shape,actual.shape,h_len.shape, a_ids.shape

In [ ]:
print(findLift(predics,actual,h_len,a_ids))

# Scrapbook

Things to try:

1. Batch Normalization
2. 50, 300 User Embeddings
3. Department Embeddings
4. Product Level Model Stacked with Department Level Model
5. wavenet
6. dense layer

In [ ]:
batch_size = 1000
lr = 0.00001
embedding_size = 10
batch_norm = False
dropout = 0

[[step        0]]     [[train]]     loss: 0.68945086       [[val]]     loss: 0.68476069       
[[step      100]]     [[train]]     loss: 0.52741179       [[val]]     loss: 0.52966699       
[[step      200]]     [[train]]     loss: 0.42600287       [[val]]     loss: 0.43532646       
[[step      300]]     [[train]]     loss: 0.42100239       [[val]]     loss: 0.42119279       
[[step      400]]     [[train]]     loss: 0.40935658       [[val]]     loss: 0.41433323       
[[step      500]]     [[train]]     loss: 0.40634079       [[val]]     loss: 0.40343187       
[[step      600]]     [[train]]     loss: 0.40356448       [[val]]     loss: 0.40347679       
[[step      700]]     [[train]]     loss: 0.39721517       [[val]]     loss: 0.39683794       
[[step      800]]     [[train]]     loss: 0.40219059       [[val]]     loss: 0.40324225       
[[step      900]]     [[train]]     loss: 0.39875729       [[val]]     loss: 0.39927291       
[[step     1000]]     [[train]]     loss: 0.39288183       [[val]]     loss: 0.41149075       
[[step     1100]]     [[train]]     loss: 0.4064956        [[val]]     loss: 0.39715582       
[[step     1200]]     [[train]]     loss: 0.39463545       [[val]]     loss: 0.39509431       
[[step     1300]]     [[train]]     loss: 0.40247756       [[val]]     loss: 0.41126509       
[[step     1400]]     [[train]]     loss: 0.39764947       [[val]]     loss: 0.39076713       
[[step     1500]]     [[train]]     loss: 0.40276493       [[val]]     loss: 0.38258591       
[[step     1600]]     [[train]]     loss: 0.38685263       [[val]]     loss: 0.39597086       
[[step     1700]]     [[train]]     loss: 0.39643141       [[val]]     loss: 0.40260221       
[[step     1800]]     [[train]]     loss: 0.39657799       [[val]]     loss: 0.39385055       
[[step     1900]]     [[train]]     loss: 0.38752659       [[val]]     loss: 0.3947884        
[[step     2000]]     [[train]]     loss: 0.39012568       [[val]]     loss: 0.40079436       
[[step     2100]]     [[train]]     loss: 0.39138237       [[val]]     loss: 0.39907276    

In [ ]:
batch_size = 512
lr = 0.00001
embedding_size = 10-50
batch_norm = False
dropout = 0

[[step        0]]     [[train]]     loss: 0.69394004       [[val]]     loss: 0.69585359       
[[step      100]]     [[train]]     loss: 0.57589115       [[val]]     loss: 0.57229453       
[[step      200]]     [[train]]     loss: 0.46916369       [[val]]     loss: 0.45808681       
[[step      300]]     [[train]]     loss: 0.43033853       [[val]]     loss: 0.44753059       
[[step      400]]     [[train]]     loss: 0.41173015       [[val]]     loss: 0.40878801       
[[step      500]]     [[train]]     loss: 0.40163584       [[val]]     loss: 0.40868014       
[[step      600]]     [[train]]     loss: 0.411918         [[val]]     loss: 0.40809412       
[[step      700]]     [[train]]     loss: 0.4154951        [[val]]     loss: 0.39677772       
[[step      800]]     [[train]]     loss: 0.40657057       [[val]]     loss: 0.3949514        
[[step      900]]     [[train]]     loss: 0.40774495       [[val]]     loss: 0.40948893  

In [ ]:
batch_size = 128
lr = 0.001
embedding_size = 10-50
batch_norm = False
dropout = 0

[[step        0]]     [[train]]     loss: 0.68255645       [[val]]     loss: 0.68390912       
[[step      100]]     [[train]]     loss: 0.43326256       [[val]]     loss: 0.41730649       
[[step      200]]     [[train]]     loss: 0.42293454       [[val]]     loss: 0.4055799        
[[step      300]]     [[train]]     loss: 0.42179572       [[val]]     loss: 0.40204019       
[[step      400]]     [[train]]     loss: 0.41070997       [[val]]     loss: 0.39842628       
[[step      500]]     [[train]]     loss: 0.42036336       [[val]]     loss: 0.40866262       
[[step      600]]     [[train]]     loss: 0.41237596       [[val]]     loss: 0.3960299        
[[step      700]]     [[train]]     loss: 0.40602685       [[val]]     loss: 0.39392554       
[[step      800]]     [[train]]     loss: 0.41559202       [[val]]     loss: 0.40025059       
[[step      900]]     [[train]]     loss: 0.41720033       [[val]]     loss: 0.38500627       
[[step     1000]]     [[train]]     loss: 0.40658416       [[val]]     loss: 0.40565673       
[[step     1100]]     [[train]]     loss: 0.40783333       [[val]]     loss: 0.39481372       
[[step     1200]]     [[train]]     loss: 0.40538956       [[val]]     loss: 0.39182611       
[[step     1300]]     [[train]]     loss: 0.41342945       [[val]]     loss: 0.39858596       
[[step     1400]]     [[train]]     loss: 0.41209739       [[val]]     loss: 0.39087869       
[[step     1500]]     [[train]]     loss: 0.41145673       [[val]]     loss: 0.39485126       
[[step     1600]]     [[train]]     loss: 0.42238395       [[val]]     loss: 0.39638283       
[[step     1700]]     [[train]]     loss: 0.40937202       [[val]]     loss: 0.39356789       
[[step     1800]]     [[train]]     loss: 0.41470608       [[val]]     loss: 0.38789571       
[[step     1900]]     [[train]]     loss: 0.41887923       [[val]]     loss: 0.39698425       
[[step     2000]]     [[train]]     loss: 0.40212174       [[val]]     loss: 0.38679843       
[[step     2100]]     [[train]]     loss: 0.41022544       [[val]]     loss: 0.39420067       
[[step     2200]]     [[train]]     loss: 0.41485526       [[val]]     loss: 0.39693822       
[[step     2300]]     [[train]]     loss: 0.40914674       [[val]]     loss: 0.39013196       
[[step     2400]]     [[train]]     loss: 0.40681492       [[val]]     loss: 0.39067219       
[[step     2500]]     [[train]]     loss: 0.4076804        [[val]]     loss: 0.39330774       
[[step     2600]]     [[train]]     loss: 0.42123778       [[val]]     loss: 0.38994598       
[[step     2700]]     [[train]]     loss: 0.41556455       [[val]]     loss: 0.39031908       
[[step     2800]]     [[train]]     loss: 0.39836752       [[val]]     loss: 0.40093884       
[[step     2900]]     [[train]]     loss: 0.40989705       [[val]]     loss: 0.38727811       
[[step     3000]]     [[train]]     loss: 0.41062794       [[val]]     loss: 0.390566         
[[step     3100]]     [[train]]     loss: 0.40107831       [[val]]     loss: 0.3911186        
[[step     3200]]     [[train]]     loss: 0.40512848       [[val]]     loss: 0.3826369        
[[step     3300]]     [[train]]     loss: 0.40562024       [[val]]     loss: 0.4026496        
[[step     3400]]     [[train]]     loss: 0.42069093       [[val]]     loss: 0.38858808       
[[step     3500]]     [[train]]     loss: 0.40914303       [[val]]     loss: 0.38865459       
[[step     3600]]     [[train]]     loss: 0.41292151       [[val]]     loss: 0.39358043       
[[step     3700]]     [[train]]     loss: 0.40138045       [[val]]     loss: 0.38675016       
[[step     3800]]     [[train]]     loss: 0.41835118       [[val]]     loss: 0.39161719       
[[step     3900]]     [[train]]     loss: 0.40322772       [[val]]     loss: 0.39639044       
[[step     4000]]     [[train]]     loss: 0.43074263       [[val]]     loss: 0.3883957        
[[step     4100]]     [[train]]     loss: 0.4156726        [[val]]     loss: 0.38980969       
[[step     4200]]     [[train]]     loss: 0.40842212       [[val]]     loss: 0.39319402       
[[step     4300]]     [[train]]     loss: 0.41640786       [[val]]     loss: 0.38108944       
[[step     4400]]     [[train]]     loss: 0.40814732       [[val]]     loss: 0.40034898       
[[step     4500]]     [[train]]     loss: 0.41581489       [[val]]     loss: 0.38978179       
[[step     4600]]     [[train]]     loss: 0.4131633        [[val]]     loss: 0.38818143       
[[step     4700]]     [[train]]     loss: 0.39764337       [[val]]     loss: 0.39419644       
[[step     4800]]     [[train]]     loss: 0.40196683       [[val]]     loss: 0.38820853       
[[step     4900]]     [[train]]     loss: 0.41307212       [[val]]     loss: 0.39067163       
[[step     5000]]     [[train]]     loss: 0.40948561       [[val]]     loss: 0.39197072       
[[step     5100]]     [[train]]     loss: 0.41734304       [[val]]     loss: 0.39139198       
[[step     5200]]     [[train]]     loss: 0.40337441       [[val]]     loss: 0.38433809       
[[step     5300]]     [[train]]     loss: 0.41370722       [[val]]     loss: 0.39362062       
[[step     5400]]     [[train]]     loss: 0.41196924       [[val]]     loss: 0.38499878       
[[step     5500]]     [[train]]     loss: 0.40916227       [[val]]     loss: 0.39047131       
[[step     8400]]     [[train]]     loss: 0.41929639       [[val]]     loss: 0.39204189 


In [ ]:
batch_size = 256
lr = 0.0001
embedding_size = 10-50
batch_norm = False
dropout = 0
final layers = 2 25 - 50

[[step        0]]     [[train]]     loss: 0.69800591       [[val]]     loss: 0.7077204        
[[step      100]]     [[train]]     loss: 0.4642874        [[val]]     loss: 0.45294539       
[[step      200]]     [[train]]     loss: 0.40891664       [[val]]     loss: 0.39687391       
[[step      300]]     [[train]]     loss: 0.41119523       [[val]]     loss: 0.40390418       
[[step      400]]     [[train]]     loss: 0.40153559       [[val]]     loss: 0.39589814       
[[step      500]]     [[train]]     loss: 0.41193385       [[val]]     loss: 0.39404101       
[[step      600]]     [[train]]     loss: 0.40311682       [[val]]     loss: 0.39565056       
[[step      700]]     [[train]]     loss: 0.40937866       [[val]]     loss: 0.39157548       
[[step      800]]     [[train]]     loss: 0.40068859       [[val]]     loss: 0.39049928       
[[step      900]]     [[train]]     loss: 0.40969506       [[val]]     loss: 0.39396498       
[[step     1000]]     [[train]]     loss: 0.39421114       [[val]]     loss: 0.3906278        
[[step     1100]]     [[train]]     loss: 0.40498208       [[val]]     loss: 0.3875514        
[[step     1200]]     [[train]]     loss: 0.40884323       [[val]]     loss: 0.39262184       
[[step     1300]]     [[train]]     loss: 0.40473363       [[val]]     loss: 0.38881227       
[[step     1400]]     [[train]]     loss: 0.39670036       [[val]]     loss: 0.3909938        
[[step     1500]]     [[train]]     loss: 0.39192327       [[val]]     loss: 0.38764418       
[[step     1600]]     [[train]]     loss: 0.40501846       [[val]]     loss: 0.38803977       
[[step     1700]]     [[train]]     loss: 0.39387804       [[val]]     loss: 0.39030977       
[[step     1800]]     [[train]]     loss: 0.39960696       [[val]]     loss: 0.38569073       
[[step     1900]]     [[train]]     loss: 0.40731614       [[val]]     loss: 0.38400814       
[[step     2000]]     [[train]]     loss: 0.39492307       [[val]]     loss: 0.39189795       
[[step     2100]]     [[train]]     loss: 0.40242343       [[val]]     loss: 0.38887628       
[[step     2200]]     [[train]]     loss: 0.40079135       [[val]]     loss: 0.38510605       
[[step     2300]]     [[train]]     loss: 0.40135673       [[val]]     loss: 0.3891011        
[[step     2400]]     [[train]]     loss: 0.40815621       [[val]]     loss: 0.38702249       
[[step     2500]]     [[train]]     loss: 0.40195283       [[val]]     loss: 0.38379056       
[[step     2600]]     [[train]]     loss: 0.41210029       [[val]]     loss: 0.38936703       
[[step     2700]]     [[train]]     loss: 0.39930949       [[val]]     loss: 0.38703894       
[[step     2800]]     [[train]]     loss: 0.40331845       [[val]]     loss: 0.38391135       
[[step     2900]]     [[train]]     loss: 0.4059205        [[val]]     loss: 0.38919657       
[[step     3000]]     [[train]]     loss: 0.39222894       [[val]]     loss: 0.38493212       
[[step     3100]]     [[train]]     loss: 0.39660685       [[val]]     loss: 0.3870271        
[[step     3200]]     [[train]]     loss: 0.39953087       [[val]]     loss: 0.38373316       
[[step     3300]]     [[train]]     loss: 0.40271052       [[val]]     loss: 0.38703135       
[[step     3400]]     [[train]]     loss: 0.39658011       [[val]]     loss: 0.38717001       
[[step     3500]]     [[train]]     loss: 0.40102986       [[val]]     loss: 0.38259657       
[[step     3600]]     [[train]]     loss: 0.40681617       [[val]]     loss: 0.3822414        
[[step     3700]]     [[train]]     loss: 0.39806747       [[val]]     loss: 0.38907532      

In [ ]:

batch_size = 10000
lr = 0.001
embedding_size = 10-50
batch_norm = False
dropout = 0
final layers = 2 100 - 500
lstm = 600

[[step        0]]     [[train]]     loss: 0.65691781       [[val]]     loss: 0.65873712       
[[step      100]]     [[train]]     loss: 0.40820402       [[val]]     loss: 0.41145359       
[[step      200]]     [[train]]     loss: 0.38525348       [[val]]     loss: 0.38974458       
[[step      300]]     [[train]]     loss: 0.3921014        [[val]]     loss: 0.38808708       
[[step      400]]     [[train]]     loss: 0.3891164        [[val]]     loss: 0.39563058       
[[step      500]]     [[train]]     loss: 0.38700464       [[val]]     loss: 0.38803441       
[[step      600]]     [[train]]     loss: 0.38530519       [[val]]     loss: 0.38530802       
[[step      700]]     [[train]]     loss: 0.3820014        [[val]]     loss: 0.38732941       
[[step      800]]     [[train]]     loss: 0.38262708       [[val]]     loss: 0.38449259       
[[step      900]]     [[train]]     loss: 0.38381389       [[val]]     loss: 0.38279317       
[[step     1000]]     [[train]]     loss: 0.38220652       [[val]]     loss: 0.38621109       
[[step     1100]]     [[train]]     loss: 0.38452308       [[val]]     loss: 0.38545803       
[[step     1200]]     [[train]]     loss: 0.38348263       [[val]]     loss: 0.38144997       
[[step     1300]]     [[train]]     loss: 0.38537586       [[val]]     loss: 0.38501431       
[[step     1400]]     [[train]]     loss: 0.38123322       [[val]]     loss: 0.38310651       
[[step     1500]]     [[train]]     loss: 0.38254252       [[val]]     loss: 0.38495605       
[[step     1600]]     [[train]]     loss: 0.37771166       [[val]]     loss: 0.38185424       
[[step     1700]]     [[train]]     loss: 0.38058311       [[val]]     loss: 0.38246381       
[[step     1800]]     [[train]]     loss: 0.38418386       [[val]]     loss: 0.38565543       
[[step     1900]]     [[train]]     loss: 0.3772252        [[val]]     loss: 0.37936874       
[[step     2000]]     [[train]]     loss: 0.38489054       [[val]]     loss: 0.38112491       
[[step     2100]]     [[train]]     loss: 0.37931004       [[val]]     loss: 0.38886817       
[[step     2200]]     [[train]]     loss: 0.38216422       [[val]]     loss: 0.38252707       
[[step     2300]]     [[train]]     loss: 0.38276958       [[val]]     loss: 0.38094901       
[[step     2400]]     [[train]]     loss: 0.3831657        [[val]]     loss: 0.38399107       
[[step     2500]]     [[train]]     loss: 0.38187274       [[val]]     loss: 0.38105462       
[[step     2600]]     [[train]]     loss: 0.38516242       [[val]]     loss: 0.38006142       
[[step     2700]]     [[train]]     loss: 0.38071658       [[val]]     loss: 0.3846091        
[[step     2800]]     [[train]]     loss: 0.38133765       [[val]]     loss: 0.38212655       
[[step     2900]]     [[train]]     loss: 0.37816642       [[val]]     loss: 0.37891927       
[[step     3000]]     [[train]]     loss: 0.38419693       [[val]]     loss: 0.38428155       
[[step     3100]]     [[train]]     loss: 0.3781649        [[val]]     loss: 0.38119657       
[[step     3200]]     [[train]]     loss: 0.38513189       [[val]]     loss: 0.38320454       
[[step     3300]]     [[train]]     loss: 0.38730632       [[val]]     loss: 0.37999186       
[[step     3400]]     [[train]]     loss: 0.38231762       [[val]]     loss: 0.38117058       
[[step     3500]]     [[train]]     loss: 0.3831878        [[val]]     loss: 0.38359242       
[[step     3600]]     [[train]]     loss: 0.38420483       [[val]]     loss: 0.37936406       
[[step     3700]]     [[train]]     loss: 0.38203219       [[val]]     loss: 0.37749232       
[[step     3800]]     [[train]]     loss: 0.37881707       [[val]]     loss: 0.38570269       
[[step     3900]]     [[train]]     loss: 0.38056242       [[val]]     loss: 0.38261534       
[[step     4000]]     [[train]]     loss: 0.37818603       [[val]]     loss: 0.37869769       
[[step     4100]]     [[train]]     loss: 0.37891285       [[val]]     loss: 0.38342259       
[[step     4200]]     [[train]]     loss: 0.37394667       [[val]]     loss: 0.38115527       
[[step     4300]]     [[train]]     loss: 0.37930212       [[val]]     loss: 0.37814815       
[[step     4400]]     [[train]]     loss: 0.37921889       [[val]]     loss: 0.3847813        
[[step     4500]]     [[train]]     loss: 0.37409336       [[val]]     loss: 0.38351414       
[[step     4600]]     [[train]]     loss: 0.37662914       [[val]]     loss: 0.38074349       
[[step     4700]]     [[train]]     loss: 0.37437559       [[val]]     loss: 0.38719823       
[[step     4800]]     [[train]]     loss: 0.37528483       [[val]]     loss: 0.3823637        
[[step     4900]]     [[train]]     loss: 0.37472385       [[val]]     loss: 0.38422586       
[[step     5000]]     [[train]]     loss: 0.37233163       [[val]]     loss: 0.38127384       
[[step     5100]]     [[train]]     loss: 0.37854063       [[val]]     loss: 0.38465769       
[[step     5200]]     [[train]]     loss: 0.3731164        [[val]]     loss: 0.38532945       
[[step     5300]]     [[train]]     loss: 0.37532851       [[val]]     loss: 0.38037309       
[[step     5400]]     [[train]]     loss: 0.37103678       [[val]]     loss: 0.38014162       
[[step     5500]]     [[train]]     loss: 0.37317753       [[val]]     loss: 0.38685308       
[[step     5600]]     [[train]]     loss: 0.36903966       [[val]]     loss: 0.38514764       
[[step     5700]]     [[train]]     loss: 0.3736941        [[val]]     loss: 0.38151485       
[[step     5800]]     [[train]]     loss: 0.37574548       [[val]]     loss: 0.38607571       
[[step     5900]]     [[train]]     loss: 0.36797815       [[val]]     loss: 0.38232391       
[[step     6000]]     [[train]]     loss: 0.37378719       [[val]]     loss: 0.37931299       
[[step     6100]]     [[train]]     loss: 0.37145472       [[val]]     loss: 0.3854492        
[[step     6200]]     [[train]]     loss: 0.37422799       [[val]]     loss: 0.38575146       
[[step     6300]]     [[train]]     loss: 0.37489267       [[val]]     loss: 0.3798244        
[[step     6400]]     [[train]]     loss: 0.37317946       [[val]]     loss: 0.3865814        
[[step     6500]]     [[train]]     loss: 0.37357524       [[val]]     loss: 0.38192061       
[[step     6600]]     [[train]]     loss: 0.37510183       [[val]]     loss: 0.38409594       
[[step     6700]]     [[train]]     loss: 0.37295555       [[val]]     loss: 0.38225858       
[[step     6800]]     [[train]]     loss: 0.37292404       [[val]]     loss: 0.38288011       
[[step     6900]]     [[train]]     loss: 0.37217351       [[val]]     loss: 0.38604897       
[[step     7000]]     [[train]]     loss: 0.37124587       [[val]]     loss: 0.37983757       
[[step     7100]]     [[train]]     loss: 0.3734486        [[val]]     loss: 0.38058637       
[[step     7200]]     [[train]]     loss: 0.37438173       [[val]]     loss: 0.38803061       
[[step     7300]]     [[train]]     loss: 0.37942499       [[val]]     loss: 0.38332724       
[[step     7400]]     [[train]]     loss: 0.37363714       [[val]]     loss: 0.38279361       
[[step     7500]]     [[train]]     loss: 0.37495664       [[val]]     loss: 0.38467916       
[[step     7600]]     [[train]]     loss: 0.37496262       [[val]]     loss: 0.38225796       
[[step     7700]]     [[train]]     loss: 0.37258991       [[val]]     loss: 0.37846636       
[[step     7800]]     [[train]]     loss: 0.37120649       [[val]]     loss: 0.38686409       
[[step     7900]]     [[train]]     loss: 0.37128173       [[val]]     loss: 0.38571515       
[[step     8000]]     [[train]]     loss: 0.37181711       [[val]]     loss: 0.37960169       
[[step     8100]]     [[train]]     loss: 0.36826489       [[val]]     loss: 0.38574902       
[[step     8200]]     [[train]]     loss: 0.37130566       [[val]]     loss: 0.38235267       
[[step     8300]]     [[train]]     loss: 0.37132396       [[val]]     loss: 0.3838576        
[[step     8400]]     [[train]]     loss: 0.36927456       [[val]]     loss: 0.38404872       
[[step     8500]]     [[train]]     loss: 0.36493352       [[val]]     loss: 0.3866663        
[[step     8600]]     [[train]]     loss: 0.36904615       [[val]]     loss: 0.38660518       
[[step     8700]]     [[train]]     loss: 0.36574878       [[val]]     loss: 0.382791         
[[step     8800]]     [[train]]     loss: 0.36782915       [[val]]     loss: 0.38435617       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     8900]]     [[train]]     loss: 0.36883847       [[val]]     loss: 0.38830326       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9000]]     [[train]]     loss: 0.36762432       [[val]]     loss: 0.38375869       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9100]]     [[train]]     loss: 0.372115         [[val]]     loss: 0.38439275       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9200]]     [[train]]     loss: 0.36688577       [[val]]     loss: 0.38790736       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9300]]     [[train]]     loss: 0.36813599       [[val]]     loss: 0.38178434       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9400]]     [[train]]     loss: 0.36671974       [[val]]     loss: 0.37995047       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9500]]     [[train]]     loss: 0.36658081       [[val]]     loss: 0.38970799       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9600]]     [[train]]     loss: 0.36533744       [[val]]     loss: 0.38604101       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9700]]     [[train]]     loss: 0.36672812       [[val]]     loss: 0.38222775       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9800]]     [[train]]     loss: 0.36806024       [[val]]     loss: 0.38704535       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step     9900]]     [[train]]     loss: 0.3634537        [[val]]     loss: 0.38424529       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.
[[step    10000]]     [[train]]     loss: 0.36658145       [[val]]     loss: 0.38508767       
best validation loss of 0.377492317557 at training step 3700
early stopping - ending training.